## Project: Can Technical Indicator-based Stock Trading Strategies Perform Better than Top-performing Mutual Funds?

### Trend Following Strategy

**Moving Average Crossover:**  
Calculates 50-day and 200-day moving averages. Generates buy signals when the short MA crosses above the long MA, and sell signals when it crosses below. Extracts trades and computes performance metrics.


In [ ]:
# Import necessary libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Define the list of stock tickers and date range
holdings = ["NVDA", "AAPL", "MSFT", "GOOGL", "AVGO", "AMZN", "TSLA", "META", "NFLX", "PLTR"]
start_date = "2015-01-01"
end_date = "2025-10-31"


In [ ]:
# Function to calculate buy/sell trades and returns
def calculate_trades(data):
    """
    Calculate buy/sell trades based on the Position column.

    Args:
        data (DataFrame): DataFrame containing 'Position' and 'Close' columns.

    Returns:
        DataFrame: Trade details (Buy Date, Sell Date, Buy Price, Sell Price, Return %).
    """
    trades = []
    buy_date, buy_price = None, None

    for i in range(len(data)):
        if data['Position'].iloc[i] == 1 and buy_date is None:  # Buy signal
            buy_date = data.index[i]
            buy_price = data['Close'].iloc[i]
        elif data['Position'].iloc[i] == -1 and buy_date is not None:  # Sell signal
            sell_date = data.index[i]
            sell_price = data['Close'].iloc[i]
            return_cal = ((sell_price - buy_price) / buy_price) 
            trades.append({
                "Buy Date": buy_date.date().strftime("%d-%m-%Y"),
                "Sell Date": sell_date.date().strftime("%d-%m-%Y"),
                "Buy Price": buy_price,
                "Sell Price": sell_price,
                "Return": return_cal
            })
            buy_date, buy_price = None, None  # Reset after a trade
    return pd.DataFrame(trades)

In [ ]:
#calculate moving averages and trades for each stock
ma_all_trades = []
ma_dict = {}
data = yf.download(holdings, start=start_date, end=end_date, interval="1d", group_by="ticker")

# calculate moving averages and trades for each stock
for ticker in holdings:
    print(f"Processing {ticker}...")
    try:
        stock_ma = data[ticker][['Close']].dropna().copy()
        stock_ma['Short MA'] = stock_ma['Close'].rolling(window=50, min_periods=1).mean()
        stock_ma['Long MA'] = stock_ma['Close'].rolling(window=200, min_periods=1).mean()
        stock_ma['Position'] = 0
        stock_ma.loc[stock_ma.index[200:],'Position'] = np.where(stock_ma['Short MA'][200:] > stock_ma['Long MA'][200:], 1, -1)
                
        ma_dict[ticker] = stock_ma[['Close', 'Short MA', 'Long MA']].copy()
        trades = calculate_trades(stock_ma)
        if not trades.empty:
            trades['Ticker'] = ticker
            ma_all_trades.append(trades)
    except KeyError as e:
        print(f"Error processing {ticker}: {e}")
        continue

ma_all_trades_df = pd.concat(ma_all_trades, ignore_index=True)
if not ma_all_trades_df.empty:
    columns = ['Ticker'] + [col for col in ma_all_trades_df.columns if col != 'Ticker']
    ma_all_trades_df = ma_all_trades_df[columns]


In [ ]:
#Ensure "Sell Date" is datetime and extract year
ma_all_trades_df['Sell Date'] = pd.to_datetime(ma_all_trades_df['Sell Date'], format='%d-%m-%Y', errors='coerce')
ma_all_trades_df['Buy Date'] = pd.to_datetime(ma_all_trades_df['Buy Date'], format='%d-%m-%Y', errors='coerce')
ma_all_trades_df['Year'] = ma_all_trades_df['Sell Date'].dt.year

In [ ]:
#define metrics calculation function
def calculate_metrics(trades_df, risk_free_rate=0.02):
    """
    Calculate performance, risk, and risk-adjusted metrics for a single ticker.

    Args:
        trades_df (DataFrame): Trade details for the ticker.
        ticker (str): Stock ticker symbol.
        risk_free_rate (float): Annual risk-free rate (default is 2%).

    Returns:
        dict: All calculated metrics for the ticker.
    """
    if trades_df.empty:
        return pd.DataFrame([{
            "Ticker": trades_df['Ticker'].iloc[0] if not trades_df.empty else np.nan,
            "Year" : trades_df['Year'].iloc[0] if not trades_df.empty else np.nan,
            "CAGR": np.nan,
            "Total Return": np.nan,
            "Standard Return": np.nan,
            "Cumulative Return": np.nan,
            "Volatility": np.nan,
            "Max Drawdown": np.nan,
            "Sharpe Ratio": np.nan,
            "Win Rate (%)": np.nan,
            "Profit Factor": np.nan
        }])

    trades_df = trades_df.copy()
    trades_df['Daily Return'] = trades_df['Return']  # <-- Fix is here

    total_return = (trades_df['Sell Price'].sum() - trades_df['Buy Price'].sum()) / trades_df['Buy Price'].sum()
    start_date = trades_df['Buy Date'].min()
    end_date = trades_df['Sell Date'].max()
    total_years = (end_date - start_date).days / 365.25 if (end_date - start_date).days > 0 else 1
    cagr = (1 + total_return) ** (1 / total_years) - 1 if total_years > 0 else np.nan
    

    standard_return = trades_df['Return'] + 1
    cumulative_return = standard_return.cumprod() - 1
    
    # Only store the final value for each
    final_standard_return = standard_return.iloc[-1] if not standard_return.empty else np.nan
    final_cumulative_return = cumulative_return.iloc[-1] if not cumulative_return.empty else np.nan

    daily_volatility = trades_df['Daily Return'].std()
    annualized_volatility = daily_volatility * np.sqrt(252)

    rolling_max = cumulative_return.cummax()
    drawdown = (cumulative_return - rolling_max) / rolling_max.replace(0, np.nan)
    max_drawdown = drawdown.min()

    sharpe_ratio = (cagr - risk_free_rate) / annualized_volatility if annualized_volatility != 0 else np.nan
    win_rate = (trades_df['Return'] > 0).mean() * 100
    gross_profit = trades_df.loc[trades_df['Return'] > 0, 'Return'].sum()
    gross_loss = -trades_df.loc[trades_df['Return'] < 0, 'Return'].sum()
    profit_factor = gross_profit / gross_loss if gross_loss != 0 else np.nan

    return pd.DataFrame([{
        "Ticker": trades_df['Ticker'].iloc[0],
        "Year": trades_df['Year'].iloc[0],
        "CAGR": cagr,
        "Total Return": total_return,
        "Standard Return": final_standard_return,
        "Cumulative Return": final_cumulative_return,
        "Volatility": annualized_volatility,
        "Max Drawdown": max_drawdown,
        "Sharpe Ratio": sharpe_ratio,
        "Win Rate (%)": win_rate,
        "Profit Factor": profit_factor
    }])



In [ ]:
#group by ticker and year to calculate annual metrics for each group
ma_metrics_by_year = ma_all_trades_df.groupby(['Ticker', 'Year']).apply(calculate_metrics, risk_free_rate=0.02).reset_index(drop=True)

#Calculate overall metrics for each ticker
ma_metrics_rows = []
for ticker in holdings:
    ma_ticker_trades = ma_all_trades_df[ma_all_trades_df['Ticker'] == ticker]
    ma_metrics_row = calculate_metrics(ma_ticker_trades)
    ma_metrics_rows.append(ma_metrics_row)
ma_metrics_df = pd.concat(ma_metrics_rows, ignore_index=True)

# 9. Display or plot results as needed
print(ma_metrics_by_year.head())
print(ma_metrics_df.head())

In [ ]:
# sorting values
ma_metrics_by_year.sort_values(by=['Ticker', 'Year'], inplace=True)
ma_metrics_by_year.head()

In [ ]:
# Plot moving averages for each stock
for ticker in ma_dict:
    stock_ma_plot = ma_dict[ticker].copy()

    # Get buy/sell signals from MA strategy
    trades_ma = ma_all_trades_df[ma_all_trades_df['Ticker'] == ticker]
    buy_signals_ma = stock_ma_plot.loc[stock_ma_plot.index.isin(trades_ma['Buy Date'])]
    sell_signals_ma = stock_ma_plot.loc[stock_ma_plot.index.isin(trades_ma['Sell Date'])]
    
   
    fig, axs = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    sns.set_style("whitegrid")
    # Top plot: CLose Price + buy/sell signals
    axs[0].plot(stock_ma_plot.index, stock_ma_plot['Close'], label='Close Price', color='blue', alpha=0.7)
    axs[0].scatter(buy_signals_ma.index, buy_signals_ma['Close'], label='Buy Signal', marker='^', color='green', s=100)
    axs[0].scatter(sell_signals_ma.index, sell_signals_ma['Close'], label='Sell Signal', marker='v', color='red', s=100)
    axs[0].set_ylabel("Price")
    axs[0].set_title(f"Moving Averages with Buy/Sell Signals for {ticker}")
    axs[0].legend()
    axs[0].grid(True, linestyle='--', alpha=0.5)

    # Bottom plot: Moving Averages
    axs[1].plot(stock_ma_plot.index, stock_ma_plot['Short MA'], label='50-Day MA', color='orange')
    axs[1].plot(stock_ma_plot.index, stock_ma_plot['Long MA'], label='200-Day MA', color='green')
    axs[1].set_ylabel("Moving Averages")
    axs[1].set_title(f"Moving Averages for {ticker}")
    axs[1].legend()
    axs[1].grid(True, linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.show()


In [ ]:

# Get tickers from ma_dict
tickers = list(ma_dict.keys())
n_tickers = len(tickers)

sns.set_style("whitegrid")
fig, axes = plt.subplots(n_tickers * 2, 1, figsize=(18, 5 * n_tickers), sharex=False)
axes = axes.flatten()

for i, ticker in enumerate(tickers):
    stock = ma_dict[ticker].copy()
    trades = ma_all_trades_df[ma_all_trades_df['Ticker'] == ticker]
    buy_signals = stock.loc[stock.index.isin(trades['Buy Date'])]
    sell_signals = stock.loc[stock.index.isin(trades['Sell Date'])]

    ax_top = axes[i * 2]
    ax_bottom = axes[i * 2 + 1]

    # Top: Close price + buy/sell signals
    ax_top.plot(stock.index, stock['Close'], label='Close Price', color='blue', alpha=0.7)
    ax_top.scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy', s=80)
    ax_top.scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell', s=80)
    ax_top.set_ylabel("Price")
    ax_top.set_title(f"{ticker} MA Strategy Trades")
    ax_top.legend()
    ax_top.grid(True, linestyle='--', alpha=0.5)

    # Bottom: MA lines only
    ax_bottom.plot(stock.index, stock['Short MA'], label='50-Day MA', color='orange')
    ax_bottom.plot(stock.index, stock['Long MA'], label='200-Day MA', color='green')
    ax_bottom.set_ylabel("MA Value")
    ax_bottom.set_xlabel("Date")
    ax_bottom.set_title(f"{ticker} 50-Day & 200-Day Moving Averages")
    ax_bottom.legend()
    ax_bottom.grid(True, linestyle='--', alpha=0.5)

plt.suptitle('Moving Average Strategy - All Tickers', fontsize=18, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# Extract year from date for annual grouping
ma_metrics_by_year['Year'] = ma_metrics_by_year['Year']
annual_return = ma_metrics_by_year.groupby(by='Year')['Standard Return'].prod()-1
# Plot annual returns as a bar char
annual_return.plot(kind='bar')
plt.show()

In [ ]:
# calculate pivot table for total returns by ticker and year
total_return_ma = ma_metrics_by_year.pivot(index='Ticker', columns='Year', values='Total Return')
total_return_ma

In [ ]:
# pivot table for MA strategy
ma_pivot = ma_metrics_by_year.groupby('Year')['Total Return'].mean().to_frame().T
ma_pivot.index = ['MA Strategy']
ma_pivot

In [ ]:
# Bar plot: Total Return by Year (all tickers combined, mean)
total_return_by_year_ma = ma_metrics_by_year.groupby('Year')['Total Return'].mean()

plt.figure(figsize=(10, 5))
plt.bar(total_return_by_year_ma.index, total_return_by_year_ma.values, color='skyblue')
plt.ylabel('Average Total Return')
plt.xlabel('Year')
plt.title('Average Total Return by Year (All Tickers)')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(18, 14))
fig.suptitle('MA Strategy Metrics', fontsize=16)

# Cumulative Return by Ticker
axs[0, 0].bar(ma_metrics_df['Ticker'], ma_metrics_df['Cumulative Return'], color='mediumseagreen')
axs[0, 0].set_title('Cumulative Return')
axs[0, 0].set_xlabel('Ticker')
axs[0, 0].set_ylabel('Cumulative Return')
axs[0, 0].grid(axis='y', linestyle='--', alpha=0.7)

# CAGR by Ticker
axs[0, 1].bar(ma_metrics_df['Ticker'], ma_metrics_df['CAGR'], color='royalblue')
axs[0, 1].set_title('CAGR')
axs[0, 1].set_xlabel('Ticker')
axs[0, 1].set_ylabel('CAGR')
axs[0, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Total Return by Ticker
axs[0, 2].bar(ma_metrics_df['Ticker'], ma_metrics_df['Total Return'], color='seagreen')
axs[0, 2].set_title('Total Return')
axs[0, 2].set_xlabel('Ticker')
axs[0, 2].set_ylabel('Total Return')
axs[0, 2].grid(axis='y', linestyle='--', alpha=0.7)

# Standard Return by Ticker
axs[1, 0].bar(ma_metrics_df['Ticker'], ma_metrics_df['Standard Return'], color='slateblue')
axs[1, 0].set_title('Standard Return')
axs[1, 0].set_xlabel('Ticker')
axs[1, 0].set_ylabel('Standard Return')
axs[1, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Volatility by Ticker
axs[1, 1].bar(ma_metrics_df['Ticker'], ma_metrics_df['Volatility'], color='slateblue')
axs[1, 1].set_title('Volatility')
axs[1, 1].set_xlabel('Ticker')
axs[1, 1].set_ylabel('Volatility')
axs[1, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Sharpe Ratio by Ticker
axs[1, 2].bar(ma_metrics_df['Ticker'], ma_metrics_df['Sharpe Ratio'], color='orange')
axs[1, 2].set_title('Sharpe Ratio')
axs[1, 2].set_xlabel('Ticker')
axs[1, 2].set_ylabel('Sharpe Ratio')
axs[1, 2].grid(axis='y', linestyle='--', alpha=0.7)

# Max Drawdown by Ticker
axs[2, 0].bar(ma_metrics_df['Ticker'], ma_metrics_df['Max Drawdown'], color='crimson')
axs[2, 0].set_title('Max Drawdown')
axs[2, 0].set_xlabel('Ticker')
axs[2, 0].set_ylabel('Max Drawdown')
axs[2, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Win Rate (%) by Ticker
axs[2, 1].bar(ma_metrics_df['Ticker'], ma_metrics_df['Win Rate (%)'], color='purple')
axs[2, 1].set_title('Win Rate (%)')
axs[2, 1].set_xlabel('Ticker')
axs[2, 1].set_ylabel('Win Rate (%)')
axs[2, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Profit Factor by Ticker
axs[2, 2].bar(ma_metrics_df['Ticker'], ma_metrics_df['Profit Factor'], color='teal')
axs[2, 2].set_title('Profit Factor')
axs[2, 2].set_xlabel('Ticker')
axs[2, 2].set_ylabel('Profit Factor')
axs[2, 2].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
tik = yf.Ticker("0P000071W5.TO") # Fund ticker - TD Canadian Balanced Fund
fund_data = tik.history(start="2015-01-01", end="2025-10-31", interval="1d")
fund_data.head()

In [ ]:
# 1. Calculate average annual return for your MA strategy
ma_annual_return = ma_metrics_by_year.groupby('Year')['Total Return'].mean()

# 2. Calculate annual return for the mutual fund
fund_data['Fund Return'] = fund_data['Close'].pct_change()
fund_data = fund_data.dropna(subset=['Fund Return'])
fund_data['Year'] = fund_data.index.year
fund_annual_return = fund_data.groupby('Year')['Fund Return'].apply(lambda x: (1 + x).prod() - 1)

# 3. Plot comparison
plt.figure(figsize=(12, 6))
plt.bar(ma_annual_return.index - 0.15, ma_annual_return.values, width=0.3, label='MA Strategy Avg Return', color='skyblue')
plt.bar(fund_annual_return.index + 0.15, fund_annual_return.values, width=0.3, label='Mutual Fund Return', color='orange')
plt.xlabel('Year')
plt.ylabel('Annual Return')
plt.title('Annual Return: MA Strategy (Avg) vs. Mutual Fund')
plt.legend()
plt.xticks(sorted(set(ma_annual_return.index) | set(fund_annual_return.index)), rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# --- Summary Table: MA Strategy (Avg) vs. Mutual Fund ---

# MA Strategy (average across tickers)
ma_summary = pd.DataFrame({
    'Strategy': ['MA Strategy (Avg)'],
    'CAGR': [ma_metrics_df['CAGR'].mean()],
    'Total Return': [ma_metrics_df['Total Return'].mean()],
    'Volatility': [ma_metrics_df['Volatility'].mean()],
    'Max Drawdown': [ma_metrics_df['Max Drawdown'].mean()],
    'Sharpe Ratio': [ma_metrics_df['Sharpe Ratio'].mean()],
    'Win Rate (%)': [ma_metrics_df['Win Rate (%)'].mean()],
    'Profit Factor': [ma_metrics_df['Profit Factor'].mean()]
})

# Mutual Fund (using fund_data)
fund_data['Fund Return'] = fund_data['Close'].pct_change()
fund_data = fund_data.dropna(subset=['Fund Return'])
fund_cum_return = (1 + fund_data['Fund Return']).prod() - 1
fund_years = (fund_data.index[-1] - fund_data.index[0]).days / 365.25
fund_cagr = (1 + fund_cum_return) ** (1 / fund_years) - 1 if fund_years > 0 else np.nan
fund_volatility = fund_data['Fund Return'].std() * np.sqrt(252)
fund_rolling = (1 + fund_data['Fund Return']).cumprod() - 1
fund_drawdown = (fund_rolling - fund_rolling.cummax()) / fund_rolling.cummax().replace(0, np.nan)
fund_max_drawdown = fund_drawdown.min()
fund_sharpe = (fund_cagr - 0.02) / fund_volatility if fund_volatility != 0 else np.nan

fund_win_rate = (fund_data['Fund Return'] > 0).mean() * 100
fund_gross_profit = fund_data.loc[fund_data['Fund Return'] > 0, 'Fund Return'].sum()
fund_gross_loss = -fund_data.loc[fund_data['Fund Return'] < 0, 'Fund Return'].sum()
fund_profit_factor = fund_gross_profit / fund_gross_loss if fund_gross_loss != 0 else np.nan

fund_summary = pd.DataFrame({
    'Strategy': ['Mutual Fund'],
    'CAGR': [fund_cagr],
    'Total Return': [fund_cum_return],
    'Volatility': [fund_volatility],
    'Max Drawdown': [fund_max_drawdown],
    'Sharpe Ratio': [fund_sharpe],
    'Win Rate (%)': [fund_win_rate],
    'Profit Factor': [fund_profit_factor]
})

# 3. Combine and display
summary_table = pd.concat([ma_summary, fund_summary], ignore_index=True)
summary_table.set_index('Strategy', inplace=True)
display(summary_table)

In [ ]:
# Plot CAGR for MA Strategy (Avg) vs Mutual Fund
summary_table_plot = summary_table.reset_index()
plt.figure(figsize=(6, 5))

strategies = summary_table_plot['Strategy']
cagr_values = summary_table_plot['CAGR']

plt.bar(strategies, cagr_values, color=['skyblue', 'orange'])
plt.ylabel('CAGR')
plt.title('CAGR: MA Strategy (Avg) vs. Mutual Fund')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

**Breakout Strategy:**  
Finds 20-day high/low. Buys when price breaks above the previous 20-day high, sells when it breaks below the previous 20-day low. Trades and metrics are calculated.

In [ ]:
# Breakout Strategy: Calculate trades based on 20-day high/low breakout
breakout_trades = []
breakout_dict = {}

for ticker in holdings:
    print(f"Processing {ticker} for breakout strategy...")
    try:
        stock_bo = data[ticker][['Close']].dropna().copy()
        # Calculate the 20-day high and low
        stock_bo['20 Day High'] = stock_bo['Close'].rolling(window=20, min_periods=1).max()
        stock_bo['20 Day Low'] = stock_bo['Close'].rolling(window=20, min_periods=1).min()
        stock_bo['Position'] = 0
        # Signal: 1 if price breaks above previous 20-day high, -1 if below previous 20-day low
        stock_bo['Position'] = np.where(stock_bo['Close'] > stock_bo['20 Day High'].shift(1), 1,
                                       np.where(stock_bo['Close'] < stock_bo['20 Day Low'].shift(1), -1, 0))
        # Entry/exit marker logic
        stock_bo['Hold_Position'] = 0
        current = 0
        for idx, row in stock_bo.iterrows():
            if row['Position'] == 1:
                current = 1         # Enter long position
            elif row['Position'] == -1:
                current = -1        # Enter short position
            stock_bo.at[idx, 'Hold_Position'] = current
        # Detect entry/exit points
        stock_bo['Prev_Hold'] = stock_bo['Hold_Position'].shift(1).fillna(0)
        stock_bo['Pos_Marker'] = 0

        # Mark entry and exit points for long positions
        stock_bo.loc[(stock_bo['Prev_Hold'] != 1) & (stock_bo['Hold_Position'] == 1), 'Pos_Marker'] = 1
        stock_bo.loc[(stock_bo['Prev_Hold'] == 1) & (stock_bo['Hold_Position'] != 1), 'Pos_Marker'] = -1

        breakout_dict[ticker] = stock_bo[['Close', '20 Day High', '20 Day Low', 'Position', 'Hold_Position', 'Pos_Marker']].copy()
        
        # Extract trades
        trades_bo = calculate_trades(pd.DataFrame({'Close': stock_bo['Close'], 'Position': stock_bo['Pos_Marker']}, index=stock_bo.index))
        if not trades_bo.empty:
            trades_bo['Ticker'] = ticker
            breakout_trades.append(trades_bo)
    except KeyError as e:
        print(f"Error processing {ticker} for breakout strategy: {e}")
        continue
# combine all breakout trades into a single DataFrame
breakout_trades_df = pd.concat(breakout_trades, ignore_index=True)
if not breakout_trades_df.empty:
    columns = ['Ticker'] + [col for col in breakout_trades_df.columns if col != 'Ticker']
    breakout_trades_df = breakout_trades_df[columns]

In [ ]:
#Ensure "Sell Date" is datetime and extract year
breakout_trades_df['Sell Date'] = pd.to_datetime(breakout_trades_df['Sell Date'], format='%d-%m-%Y', errors='coerce')
breakout_trades_df['Buy Date'] = pd.to_datetime(breakout_trades_df['Buy Date'], format='%d-%m-%Y', errors='coerce')
breakout_trades_df['Year'] = breakout_trades_df['Sell Date'].dt.year

In [ ]:
#group by ticker and year to calculate annual metrics for breakout strategy
metrics_bo_by_year = breakout_trades_df.groupby(['Ticker', 'Year']).apply(calculate_metrics, risk_free_rate=0.02).reset_index(drop=True)

#calculate overall metrics for each ticker for breakout strategy
metrics_rows_bo = []
for ticker in holdings:
    ticker_trades_bo = breakout_trades_df[breakout_trades_df['Ticker'] == ticker]
    metrics_row_bo = calculate_metrics(ticker_trades_bo)
    metrics_rows_bo.append(metrics_row_bo)
metrics_df_bo = pd.concat(metrics_rows_bo, ignore_index=True)

# Display or plot results as needed
print(metrics_bo_by_year.head())
print(metrics_df_bo.head())

In [ ]:
# sorting values
metrics_bo_by_year.sort_values(by=['Ticker', 'Year'], inplace=True)
metrics_bo_by_year.head()

In [ ]:
# Plot breakout strategy results for each stock
for ticker in breakout_dict:
    stock_bo_plot = breakout_dict[ticker].copy()
    # Buy and Sell signals
    buy_signals = stock_bo_plot[stock_bo_plot['Pos_Marker'] == 1]
    sell_signals = stock_bo_plot[stock_bo_plot['Pos_Marker'] == -1]
    
    # Create a figure with two subplots
    fig, axs = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    sns.set_style("whitegrid")

    # Top subplot: Close price + buy/sell signals
    axs[0].plot(stock_bo_plot.index, stock_bo_plot['Close'], label='Close Price', color='blue', alpha=0.7)
    axs[0].scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy', s=80)
    axs[0].scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell', s=80)
    axs[0].set_ylabel("Price")
    axs[0].set_title(f"{ticker} Close Price & Buy/Sell Signals (Breakout)")
    axs[0].legend()
    axs[0].grid(True, linestyle='--', alpha=0.5)

    # Bottom subplot: Breakout bands only (20-day high/low)
    axs[1].plot(stock_bo_plot.index, stock_bo_plot['20 Day High'], label='20-Day High', color='orange', linestyle='--')
    axs[1].plot(stock_bo_plot.index, stock_bo_plot['20 Day Low'], label='20-Day Low', color='green', linestyle='--')
    axs[1].set_ylabel("Band Value")
    axs[1].set_xlabel("Date")
    axs[1].set_title(f"{ticker} Breakout Bands (20-Day High/Low)")
    axs[1].legend()
    axs[1].grid(True, linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.show()

In [ ]:

tickers = list(breakout_dict.keys())
n_tickers = len(tickers)

sns.set_style("whitegrid")
fig, axes = plt.subplots(n_tickers * 2, 1, figsize=(18, 5 * n_tickers), sharex=False)
axes = axes.flatten()

for i, ticker in enumerate(tickers):
    stock = breakout_dict[ticker].copy()
    buy_signals = stock[stock['Pos_Marker'] == 1]
    sell_signals = stock[stock['Pos_Marker'] == -1]

    ax_top = axes[i * 2]
    ax_bottom = axes[i * 2 + 1]

    # Top: Close price + buy/sell signals
    ax_top.plot(stock.index, stock['Close'], label='Close Price', color='blue', alpha=0.7)
    ax_top.scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy', s=80)
    ax_top.scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell', s=80)
    ax_top.set_ylabel("Price")
    ax_top.set_title(f"{ticker} Breakout Strategy Trades")
    ax_top.legend()
    ax_top.grid(True, linestyle='--', alpha=0.5)

    # Bottom: Breakout bands only
    ax_bottom.plot(stock.index, stock['20 Day High'], label='20-Day High', color='orange', linestyle='--')
    ax_bottom.plot(stock.index, stock['20 Day Low'], label='20-Day Low', color='green', linestyle='--')
    ax_bottom.set_ylabel("Band Value")
    ax_bottom.set_xlabel("Date")
    ax_bottom.set_title(f"{ticker} Breakout Bands (20-Day High/Low)")
    ax_bottom.legend()
    ax_bottom.grid(True, linestyle='--', alpha=0.5)

plt.suptitle('Breakout Strategy - All Tickers', fontsize=18, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# Total returns pivot table for breakout strategy
total_return_bo = metrics_bo_by_year.pivot(index='Ticker', columns='Year', values='Total Return')
total_return_bo

In [ ]:
# Breakout strategy
bo_pivot = metrics_bo_by_year.groupby('Year')['Total Return'].mean().to_frame().T
bo_pivot.index = ['Breakout Strategy']
bo_pivot


In [ ]:
#bar plot: Total Return by Year (breakout strategy, all tickers combined, mean)
total_return_by_year_bo = metrics_bo_by_year.groupby('Year')['Total Return'].mean()
total_return_by_year_bo.plot(kind='bar')
plt.title("Average Total Return by Year for Breakout Strategy")
plt.xlabel("Year")
plt.ylabel("Average Total Return")
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(18, 14))
fig.suptitle('Breakout Strategy Metrics', fontsize=16)

# Cumulative Return by Ticker
axs[0, 0].bar(metrics_df_bo['Ticker'], metrics_df_bo['Cumulative Return'], color='mediumseagreen')
axs[0, 0].set_title('Cumulative Return')
axs[0, 0].set_xlabel('Ticker')
axs[0, 0].set_ylabel('Cumulative Return')
axs[0, 0].grid(axis='y', linestyle='--', alpha=0.7)

# CAGR by Ticker
axs[0, 1].bar(metrics_df_bo['Ticker'], metrics_df_bo['CAGR'], color='royalblue')
axs[0, 1].set_title('CAGR')
axs[0, 1].set_xlabel('Ticker')
axs[0, 1].set_ylabel('CAGR')
axs[0, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Total Return by Ticker
axs[0, 2].bar(metrics_df_bo['Ticker'], metrics_df_bo['Total Return'], color='seagreen')
axs[0, 2].set_title('Total Return')
axs[0, 2].set_xlabel('Ticker')
axs[0, 2].set_ylabel('Total Return')
axs[0, 2].grid(axis='y', linestyle='--', alpha=0.7)

# Standard Return by Ticker
axs[1, 0].bar(metrics_df_bo['Ticker'], metrics_df_bo['Standard Return'], color='slateblue')
axs[1, 0].set_title('Standard Return')
axs[1, 0].set_xlabel('Ticker')
axs[1, 0].set_ylabel('Standard Return')
axs[1, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Volatility by Ticker
axs[1, 1].bar(metrics_df_bo['Ticker'], metrics_df_bo['Volatility'], color='slateblue')
axs[1, 1].set_title('Volatility')
axs[1, 1].set_xlabel('Ticker')
axs[1, 1].set_ylabel('Volatility')
axs[1, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Sharpe Ratio by Ticker
axs[1, 2].bar(metrics_df_bo['Ticker'], metrics_df_bo['Sharpe Ratio'], color='orange')
axs[1, 2].set_title('Sharpe Ratio')
axs[1, 2].set_xlabel('Ticker')
axs[1, 2].set_ylabel('Sharpe Ratio')
axs[1, 2].grid(axis='y', linestyle='--', alpha=0.7)

# Max Drawdown by Ticker
axs[2, 0].bar(metrics_df_bo['Ticker'], metrics_df_bo['Max Drawdown'], color='crimson')
axs[2, 0].set_title('Max Drawdown')
axs[2, 0].set_xlabel('Ticker')
axs[2, 0].set_ylabel('Max Drawdown')
axs[2, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Win Rate (%) by Ticker
axs[2, 1].bar(metrics_df_bo['Ticker'], metrics_df_bo['Win Rate (%)'], color='purple')
axs[2, 1].set_title('Win Rate (%)')
axs[2, 1].set_xlabel('Ticker')
axs[2, 1].set_ylabel('Win Rate (%)')
axs[2, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Profit Factor by Ticker
axs[2, 2].bar(metrics_df_bo['Ticker'], metrics_df_bo['Profit Factor'], color='teal')
axs[2, 2].set_title('Profit Factor')
axs[2, 2].set_xlabel('Ticker')
axs[2, 2].set_ylabel('Profit Factor')
axs[2, 2].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
# 1. Calculate average annual return for your MA strategy
bo_annual_return = metrics_bo_by_year.groupby('Year')['Total Return'].mean()

In [ ]:
# 2. Calculate annual return for the mutual fund
fund_data['Fund Return'] = fund_data['Close'].pct_change()
fund_data = fund_data.dropna(subset=['Fund Return'])
fund_data['Year'] = fund_data.index.year
fund_annual_return = fund_data.groupby('Year')['Fund Return'].apply(lambda x: (1 + x).prod() - 1)

In [ ]:
# 3. Plot comparison
plt.figure(figsize=(12, 6))
plt.bar(bo_annual_return.index - 0.15, bo_annual_return.values, width=0.3, label='Breakout Strategy Avg Return', color='skyblue')
plt.bar(fund_annual_return.index + 0.15, fund_annual_return.values, width=0.3, label='Mutual Fund Return', color='orange')
plt.xlabel('Year')
plt.ylabel('Annual Return')
plt.title('Annual Return: Breakout Strategy (Avg) vs. Mutual Fund')
plt.legend()
plt.xticks(sorted(set(bo_annual_return.index) | set(fund_annual_return.index)), rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
#summary table for breakout strategy
bo_summary = pd.DataFrame({
    'Strategy': ['Breakout Strategy'],
    'CAGR': [metrics_df_bo['CAGR'].mean()],
    'Total Return': [metrics_df_bo['Total Return'].mean()],
    'Volatility': [metrics_df_bo['Volatility'].mean()],
    'Max Drawdown': [metrics_df_bo['Max Drawdown'].mean()],
    'Sharpe Ratio': [metrics_df_bo['Sharpe Ratio'].mean()],
    'Win Rate (%)': [metrics_df_bo['Win Rate (%)'].mean()],
    'Profit Factor': [metrics_df_bo['Profit Factor'].mean()]
})

#use fund summary
summary_table_bo = pd.concat([bo_summary, fund_summary], ignore_index=True)
summary_table_bo.set_index('Strategy', inplace=True)
display(summary_table_bo)

In [ ]:
# Plot CAGR for Breakout Strategy (Avg) vs Mutual Fund
summary_table_bo_plot = summary_table_bo.reset_index()
plt.figure(figsize=(6, 5))

strategies_bo = summary_table_bo_plot['Strategy']
cagr_values_bo = summary_table_bo_plot['CAGR']
plt.bar(strategies_bo, cagr_values_bo, color=['skyblue', 'orange'])
plt.ylabel('CAGR')
plt.title('CAGR: Breakout Strategy (Avg) vs. Mutual Fund')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

**Donchian Channel:**  
Uses 20-day high/low bands to generate buy/sell signals on breakouts. Trades are extracted and metrics computed.

In [ ]:
# Donchian Channel Strategy: Generate buy/sell signals based on 20-day high/low breakouts
donchian_trades = []
donchian_dict = {}

for ticker in holdings:
    print(f"Processing {ticker} for donchian channel strategy...")
    try:
        stock_dc = data[ticker][['Close']].dropna().copy()
        # Calculate the 20-day high and low
        stock_dc['20 Day High'] = stock_dc['Close'].rolling(window=20, min_periods=1).max()
        stock_dc['20 Day Low'] = stock_dc['Close'].rolling(window=20, min_periods=1).min()
        stock_dc['Position'] = 0
        # Signal 1: if price breaks above previous 20-day high, -1 if below previous 20-day low
        stock_dc['Position'] = np.where(stock_dc['Close'] > stock_dc['20 Day High'].shift(1), 1,
                                       np.where(stock_dc['Close'] < stock_dc['20 Day Low'].shift(1), -1, 0))
        # Entry/exit marker logic
        stock_dc['Hold_Position'] = 0
        current = 0
        for idx, row in stock_dc.iterrows():
            if row['Position'] == 1:
                current = 1
            elif row['Position'] == -1:
                current = -1
            stock_dc.at[idx, 'Hold_Position'] = current
        # Detect entry/exit points
        stock_dc['Prev_Hold'] = stock_dc['Hold_Position'].shift(1).fillna(0)
        stock_dc['Pos_Marker'] = 0
        stock_dc.loc[(stock_dc['Prev_Hold'] != 1) & (stock_dc['Hold_Position'] == 1), 'Pos_Marker'] = 1
        stock_dc.loc[(stock_dc['Prev_Hold'] == 1) & (stock_dc['Hold_Position'] != 1), 'Pos_Marker'] = -1
        
        # Store donchian channel data for plotting
        donchian_dict[ticker] = stock_dc[['Close', '20 Day High', '20 Day Low', 'Position', 'Hold_Position', 'Pos_Marker']].copy()
        # Extract trades
        trades_dc = calculate_trades(pd.DataFrame({'Close': stock_dc['Close'], 'Position': stock_dc['Pos_Marker']}, index=stock_dc.index))
        if not trades_dc.empty:
            trades_dc['Ticker'] = ticker
            donchian_trades.append(trades_dc)
    except KeyError as e:
        print(f"Error processing {ticker} for donchian channel strategy: {e}")
        continue

# Combine all donchian trades into a single DataFrame
donchian_trades_df = pd.concat(donchian_trades, ignore_index=True)
if not donchian_trades_df.empty:
    columns = ['Ticker'] + [col for col in donchian_trades_df.columns if col != 'Ticker']
    donchian_trades_df = donchian_trades_df[columns]



In [ ]:
# Plot Donchian Channel signals for each ticker
for ticker in donchian_dict:
    stock_dc_plot = donchian_dict[ticker].copy()
    buy_signals = stock_dc_plot[stock_dc_plot['Pos_Marker'] == 1]
    sell_signals = stock_dc_plot[stock_dc_plot['Pos_Marker'] == -1]

    fig, axs = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    sns.set_style("whitegrid")

    # Top subplot: Close price + buy/sell signals
    axs[0].plot(stock_dc_plot.index, stock_dc_plot['Close'], label='Close Price', color='blue', alpha=0.7)
    axs[0].scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy', s=80)
    axs[0].scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell', s=80)
    axs[0].set_ylabel("Price")
    axs[0].set_title(f"{ticker} Close Price & Buy/Sell Signals")
    axs[0].legend()
    axs[0].grid(True, linestyle='--', alpha=0.5)

    # Bottom subplot: Donchian Channel bands only (20-day high/low)
    axs[1].plot(stock_dc_plot.index, stock_dc_plot['20 Day High'], label='20-Day High', color='orange', linestyle='--')
    axs[1].plot(stock_dc_plot.index, stock_dc_plot['20 Day Low'], label='20-Day Low', color='green', linestyle='--')
    axs[1].set_ylabel("Band Value")
    axs[1].set_xlabel("Date")
    axs[1].set_title(f"{ticker} Donchian Channel Bands")
    axs[1].legend()
    axs[1].grid(True, linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.show()

In [ ]:

tickers = list(donchian_dict.keys())
n_tickers = len(tickers)

sns.set_style("whitegrid")
fig, axes = plt.subplots(n_tickers * 2, 1, figsize=(18, 5 * n_tickers), sharex=False)
axes = axes.flatten()

for i, ticker in enumerate(tickers):
    stock = donchian_dict[ticker].copy()
    buy_signals = stock[stock['Pos_Marker'] == 1]
    sell_signals = stock[stock['Pos_Marker'] == -1]

    ax_top = axes[i * 2]
    ax_bottom = axes[i * 2 + 1]

    # Top: Close price + buy/sell signals
    ax_top.plot(stock.index, stock['Close'], label='Close Price', color='blue', alpha=0.7)
    ax_top.scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy', s=80)
    ax_top.scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell', s=80)
    ax_top.set_ylabel("Price")
    ax_top.set_title(f"{ticker} Donchian Strategy Trades")
    ax_top.legend()
    ax_top.grid(True, linestyle='--', alpha=0.5)

    # Bottom: Donchian bands only
    ax_bottom.plot(stock.index, stock['20 Day High'], label='20-Day High', color='orange', linestyle='--')
    ax_bottom.plot(stock.index, stock['20 Day Low'], label='20-Day Low', color='green', linestyle='--')
    ax_bottom.set_ylabel("Band Value")
    ax_bottom.set_xlabel("Date")
    ax_bottom.set_title(f"{ticker} Donchian Bands (20-Day High/Low)")
    ax_bottom.legend()
    ax_bottom.grid(True, linestyle='--', alpha=0.5)

plt.suptitle('Donchian Channel Strategy - All Tickers', fontsize=18, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
#ensure dates and year
donchian_trades_df['Sell Date'] = pd.to_datetime(donchian_trades_df['Sell Date'], format='%d-%m-%Y', errors='coerce')
donchian_trades_df['Buy Date'] = pd.to_datetime(donchian_trades_df['Buy Date'], format='%d-%m-%Y', errors='coerce')
donchian_trades_df['Year'] = donchian_trades_df['Sell Date'].dt.year

In [ ]:
#calculate metrics for donchian channel strategy
metrics_dc_by_year = donchian_trades_df.groupby(['Ticker', 'Year']).apply(calculate_metrics, risk_free_rate=0.02).reset_index(drop=True)
metrics_rows_dc = []
for ticker in holdings:
    ticker_trades_dc = donchian_trades_df[donchian_trades_df['Ticker'] == ticker]
    metrics_row_dc = calculate_metrics(ticker_trades_dc)
    metrics_rows_dc.append(metrics_row_dc)
metrics_df_dc = pd.concat(metrics_rows_dc, ignore_index=True)

In [ ]:
#annual return plot for donchian channel strategy
metrics_dc_by_year['Year'] = metrics_dc_by_year['Year']
annual_return_dc = metrics_dc_by_year.groupby(by='Year')['Standard Return'].prod()-1

In [ ]:
# Plot annual returns as a bar chart
annual_return_dc.plot(kind='bar')
plt.title("Annual Returns for Donchian Channel")
plt.xlabel("Year")
plt.ylabel("Annual Return")
plt.show()

In [ ]:
# Pivot table for donchian channel strategy
pivot_table_dc = metrics_dc_by_year.pivot(index='Ticker', columns='Year', values='Total Return')
pivot_table_dc

In [ ]:
# Donchian strategy
dc_pivot = metrics_dc_by_year.groupby('Year')['Total Return'].mean().to_frame().T
dc_pivot.index = ['Donchian Channel Strategy']
dc_pivot

In [ ]:
#bar plot: Total Return by Year (donchian channel strategy, all tickers combined, mean)
total_return_by_year_dc = metrics_dc_by_year.groupby('Year')['Total Return'].mean()
total_return_by_year_dc.plot(kind='bar')
plt.title("Average Total Return by Year for Donchian Channel Strategy")
plt.xlabel("Year")
plt.ylabel("Average Total Return")
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(18, 14))
fig.suptitle('Donchian Channel Strategy Metrics', fontsize=16)

# Cumulative Return by Ticker
axs[0, 0].bar(metrics_df_dc['Ticker'], metrics_df_dc['Cumulative Return'], color='mediumseagreen')
axs[0, 0].set_title('Cumulative Return')
axs[0, 0].set_xlabel('Ticker')
axs[0, 0].set_ylabel('Cumulative Return')
axs[0, 0].grid(axis='y', linestyle='--', alpha=0.7)

# CAGR by Ticker
axs[0, 1].bar(metrics_df_dc['Ticker'], metrics_df_dc['CAGR'], color='royalblue')
axs[0, 1].set_title('CAGR')
axs[0, 1].set_xlabel('Ticker')
axs[0, 1].set_ylabel('CAGR')
axs[0, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Total Return by Ticker
axs[0, 2].bar(metrics_df_dc['Ticker'], metrics_df_dc['Total Return'], color='seagreen')
axs[0, 2].set_title('Total Return')
axs[0, 2].set_xlabel('Ticker')
axs[0, 2].set_ylabel('Total Return')
axs[0, 2].grid(axis='y', linestyle='--', alpha=0.7)

# Standard Return by Ticker
axs[1, 0].bar(metrics_df_dc['Ticker'], metrics_df_dc['Standard Return'], color='slateblue')
axs[1, 0].set_title('Standard Return')
axs[1, 0].set_xlabel('Ticker')
axs[1, 0].set_ylabel('Standard Return')
axs[1, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Volatility by Ticker
axs[1, 1].bar(metrics_df_dc['Ticker'], metrics_df_dc['Volatility'], color='slateblue')
axs[1, 1].set_title('Volatility')
axs[1, 1].set_xlabel('Ticker')
axs[1, 1].set_ylabel('Volatility')
axs[1, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Sharpe Ratio by Ticker
axs[1, 2].bar(metrics_df_dc['Ticker'], metrics_df_dc['Sharpe Ratio'], color='orange')
axs[1, 2].set_title('Sharpe Ratio')
axs[1, 2].set_xlabel('Ticker')
axs[1, 2].set_ylabel('Sharpe Ratio')
axs[1, 2].grid(axis='y', linestyle='--', alpha=0.7)

# Max Drawdown by Ticker
axs[2, 0].bar(metrics_df_dc['Ticker'], metrics_df_dc['Max Drawdown'], color='crimson')
axs[2, 0].set_title('Max Drawdown')
axs[2, 0].set_xlabel('Ticker')
axs[2, 0].set_ylabel('Max Drawdown')
axs[2, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Win Rate (%) by Ticker
axs[2, 1].bar(metrics_df_dc['Ticker'], metrics_df_dc['Win Rate (%)'], color='purple')
axs[2, 1].set_title('Win Rate (%)')
axs[2, 1].set_xlabel('Ticker')
axs[2, 1].set_ylabel('Win Rate (%)')
axs[2, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Profit Factor by Ticker
axs[2, 2].bar(metrics_df_dc['Ticker'], metrics_df_dc['Profit Factor'], color='teal')
axs[2, 2].set_title('Profit Factor')
axs[2, 2].set_xlabel('Ticker')
axs[2, 2].set_ylabel('Profit Factor')
axs[2, 2].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
# 1. Calculate average annual return for your MA strategy
dc_annual_return = metrics_dc_by_year.groupby('Year')['Total Return'].mean()

In [ ]:
# 2. Calculate annual return for the mutual fund
fund_data['Fund Return'] = fund_data['Close'].pct_change()
fund_data = fund_data.dropna(subset=['Fund Return'])
fund_data['Year'] = fund_data.index.year
fund_annual_return = fund_data.groupby('Year')['Fund Return'].apply(lambda x: (1 + x).prod() - 1)

In [ ]:
# 3. Plot comparison
plt.figure(figsize=(12, 6))
plt.bar(dc_annual_return.index - 0.15, dc_annual_return.values, width=0.3, label='Donchian Strategy Avg Return', color='skyblue')
plt.bar(fund_annual_return.index + 0.15, fund_annual_return.values, width=0.3, label='Mutual Fund Return', color='orange')
plt.xlabel('Year')
plt.ylabel('Annual Return')
plt.title('Annual Return: MA Strategy (Avg) vs. Mutual Fund')
plt.legend()
plt.xticks(sorted(set(dc_annual_return.index) | set(fund_annual_return.index)), rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
#summary table for donchian channel strategy
dc_summary = pd.DataFrame({
    'Strategy': ['Donchian Channel Strategy'],
    'CAGR': [metrics_df_dc['CAGR'].mean()],
    'Total Return': [metrics_df_dc['Total Return'].mean()],
    'Volatility': [metrics_df_dc['Volatility'].mean()],
    'Max Drawdown': [metrics_df_dc['Max Drawdown'].mean()],
    'Sharpe Ratio': [metrics_df_dc['Sharpe Ratio'].mean()],
    'Win Rate (%)': [metrics_df_dc['Win Rate (%)'].mean()],
    'Profit Factor': [metrics_df_dc['Profit Factor'].mean()]
})

In [ ]:
#use fund summary
summary_table_dc = pd.concat([dc_summary, fund_summary], ignore_index=True)
summary_table_dc.set_index('Strategy', inplace=True)
display(summary_table_dc)


In [ ]:
#combine strategy metrics for comparison
trend_startegies_summary = pd.concat([ma_summary, bo_summary, dc_summary], ignore_index=True)
trend_startegies_summary.set_index('Strategy', inplace=True)
display(trend_startegies_summary)


In [ ]:
# Combine all strategy pivots into one DataFrame
combined_pivot_ma = pd.concat([ma_pivot, bo_pivot, dc_pivot], axis=0)
display(combined_pivot_ma)

## Mean Reversion

**Bollinger Bands Mean Reversion:**  
Calculates Bollinger Bands (20-day MA ± 2 std). Buys when price reverts above the lower band, sells when price reverts below the upper band. Trades and metrics are calculated.

In [ ]:
# --- Bollinger Bands: compute bands, signals, and trades ---
bb_all_trades = []
bb_dict = {}

# Parameters (tweakable)
bb_window = 20
bb_n_std = 2

for ticker in holdings:
    print(f"Processing Bollinger for {ticker}...")
    try:
        stock_bb = data[ticker][['Close']].dropna().copy()
        # rolling mean and std
        stock_bb['BB_MA'] = stock_bb['Close'].rolling(window=bb_window, min_periods=1).mean()
        stock_bb['BB_STD'] = stock_bb['Close'].rolling(window=bb_window, min_periods=1).std(ddof=0)
        stock_bb['BB_UP'] = stock_bb['BB_MA'] + bb_n_std * stock_bb['BB_STD']
        stock_bb['BB_LOW'] = stock_bb['BB_MA'] - bb_n_std * stock_bb['BB_STD']

        # Signal logic for mean-reversion (Bollinger Reversal)
        # Approach: create entry signals only on a crossing back inside the band after being outside.
        # - Long entry (1): price was below lower band yesterday and today closes above or equal to lower band (reversion from below).
        # - Short entry (-1): price was above upper band yesterday and today closes below or equal to upper band (reversion from above).
        stock_bb['Below_Low'] = stock_bb['Close'] < stock_bb['BB_LOW']
        stock_bb['Above_Up'] = stock_bb['Close'] > stock_bb['BB_UP']

        stock_bb['Prev_Below_Low'] = stock_bb['Below_Low'].shift(1).fillna(False)
        stock_bb['Prev_Above_Up'] = stock_bb['Above_Up'].shift(1).fillna(False)
        stock_bb['Position'] = 0
        # Enter long when we were below yesterday and now we moved back inside (or equal to) the lower band
        stock_bb.loc[(stock_bb['Prev_Below_Low'] == True) & (stock_bb['Close'] >= stock_bb['BB_LOW']), 'Position'] = 1
        # Enter short (or exit long) when we were above yesterday and now moved back inside (or equal to) the upper band
        stock_bb.loc[(stock_bb['Prev_Above_Up'] == True) & (stock_bb['Close'] <= stock_bb['BB_UP']), 'Position'] = -1

        # Optional: forward-fill position to simulate holding until opposite signal:
        # If you want to hold the position until an opposite signal appears, derive a holding position:
        stock_bb['Hold_Position'] = 0
        current = 0
        for idx, row in stock_bb.iterrows():
            if row['Position'] == 1:
                current = 1
            elif row['Position'] == -1:
                current = -1
            stock_bb.at[idx, 'Hold_Position'] = current
        # Use 'Hold_Position' if you want holds; otherwise `Position` has only entry rows.
        # For compatibility with your `calculate_trades`, use entry/exit markers: set 1 for buy and -1 for sell.
        # The `calculate_trades` expects a 1 to open and -1 to close. We'll convert holding into explicit entry/exit:
        # Build entry/exit markers: when Hold changes from 0 to 1 -> buy (1); when Hold changes from 1 to -1 -> sell (-1); when 1->0 (flat) -> sell (-1)
        stock_bb['Prev_Hold'] = stock_bb['Hold_Position'].shift(1).fillna(0)
        stock_bb['Pos_Marker'] = 0
        # buy marker
        stock_bb.loc[(stock_bb['Prev_Hold'] != 1) & (stock_bb['Hold_Position'] == 1), 'Pos_Marker'] = 1
        # sell marker (when leaving a long)
        stock_bb.loc[(stock_bb['Prev_Hold'] == 1) & (stock_bb['Hold_Position'] != 1), 'Pos_Marker'] = -1

        # If you prefer trades only on immediate reversion days (without multi-day holding), you can instead use 'Position'
        # For calculate_trades we expect 1 for buy day, -1 for sell day. We'll use Pos_Marker.
        trades = calculate_trades(pd.DataFrame({
            'Close': stock_bb['Close'],
            'Position': stock_bb['Pos_Marker']
        }, index=stock_bb.index))
        if not trades.empty:
            trades['Ticker'] = ticker
            # Ensure dates and Sell/Buy Price columns exist and correct
            bb_all_trades.append(trades)
        bb_dict[ticker] = stock_bb[['Close','BB_MA','BB_UP','BB_LOW','Position','Hold_Position','Pos_Marker']].copy()
    except Exception as e:
        print(f"Skipping {ticker}, error: {e}")
        continue

# Collect into DataFrame
if bb_all_trades:
    bb_all_trades_df = pd.concat(bb_all_trades, ignore_index=True)
    # Parse date strings into datetimes similar to earlier processing
    bb_all_trades_df['Sell Date'] = pd.to_datetime(bb_all_trades_df['Sell Date'], format='%d-%m-%Y', errors='coerce')
    bb_all_trades_df['Buy Date'] = pd.to_datetime(bb_all_trades_df['Buy Date'], format='%d-%m-%Y', errors='coerce')
    bb_all_trades_df['Year'] = bb_all_trades_df['Sell Date'].dt.year
else:
    bb_all_trades_df = pd.DataFrame()

In [ ]:
#group by ticker and year to calculate annual metrics for breakout strategy
metrics_bb_by_year = bb_all_trades_df.groupby(['Ticker', 'Year']).apply(calculate_metrics, risk_free_rate=0.02).reset_index(drop=True)

In [ ]:

#calculate overall metrics for each ticker for breakout strategy
metrics_rows_bb = []
for ticker in holdings:
    ticker_trades_bb = bb_all_trades_df[bb_all_trades_df['Ticker'] == ticker]
    metrics_row_bb = calculate_metrics(ticker_trades_bb)
    metrics_rows_bb.append(metrics_row_bb)
metrics_df_bb = pd.concat(metrics_rows_bb, ignore_index=True)


print(metrics_df_bb.head())

In [ ]:

for ticker in bb_dict:
    stock_bb_plot = bb_dict[ticker].copy()
    # buy and Sell signals
    buy_signals = stock_bb_plot[stock_bb_plot['Pos_Marker'] == 1]
    sell_signals = stock_bb_plot[stock_bb_plot['Pos_Marker'] == -1]
    
    # Figure with two subplots
    fig, axs = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    sns.set_style("whitegrid")

    # Top subplot: Close price + buy/sell signals
    axs[0].plot(stock_bb_plot.index, stock_bb_plot['Close'], label='Close Price', color='blue', alpha=0.7)
    axs[0].scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy', s=80)
    axs[0].scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell', s=80)
    axs[0].set_ylabel("Price")
    axs[0].set_title(f"{ticker} Close Price & Buy/Sell Signals")
    axs[0].legend()
    axs[0].grid(True, linestyle='--', alpha=0.5)

    # Bottom subplot: Bollinger Bands only (MA, upper, lower)
    axs[1].plot(stock_bb_plot.index, stock_bb_plot['BB_MA'], label='20-Day MA', color='orange')
    axs[1].plot(stock_bb_plot.index, stock_bb_plot['BB_UP'], label='Upper Band', color='green', linestyle='--')
    axs[1].plot(stock_bb_plot.index, stock_bb_plot['BB_LOW'], label='Lower Band', color='red', linestyle='--')
    axs[1].set_ylabel("Band Value")
    axs[1].set_xlabel("Date")
    axs[1].set_title(f"{ticker} Bollinger Bands")
    axs[1].legend()
    axs[1].grid(True, linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

tickers = list(bb_dict.keys())
n_tickers = len(tickers)

sns.set_style("whitegrid")
fig, axes = plt.subplots(n_tickers * 2, 1, figsize=(18, 5 * n_tickers), sharex=False)
axes = axes.flatten()

for i, ticker in enumerate(tickers):
    stock = bb_dict[ticker].copy()
    buy_signals = stock[stock['Pos_Marker'] == 1]
    sell_signals = stock[stock['Pos_Marker'] == -1]

    ax_top = axes[i * 2]
    ax_bottom = axes[i * 2 + 1]

    # Top: Close price + buy/sell signals
    ax_top.plot(stock.index, stock['Close'], label='Close Price', color='blue', alpha=0.7)
    ax_top.scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy', s=80)
    ax_top.scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell', s=80)
    ax_top.set_ylabel("Price")
    ax_top.set_title(f"{ticker} Bollinger Bands Strategy Trades")
    ax_top.legend()
    ax_top.grid(True, linestyle='--', alpha=0.5)

    # Bottom: Bollinger bands only
    ax_bottom.plot(stock.index, stock['BB_MA'], label='20-Day MA', color='orange')
    ax_bottom.plot(stock.index, stock['BB_UP'], label='Upper Band', color='green', linestyle='--')
    ax_bottom.plot(stock.index, stock['BB_LOW'], label='Lower Band', color='red', linestyle='--')
    ax_bottom.set_ylabel("Band Value")
    ax_bottom.set_xlabel("Date")
    ax_bottom.set_title(f"{ticker} Bollinger Bands (20-Day MA/±2STD)")
    ax_bottom.legend()
    ax_bottom.grid(True, linestyle='--', alpha=0.5)

plt.suptitle('Bollinger Bands Strategy - All Tickers', fontsize=18, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# Annual return plot for Bollinger Bands strategy
metrics_bb_by_year['Year'] = metrics_bb_by_year['Year']
annual_return_bb = metrics_bb_by_year.groupby(by='Year')['Standard Return'].prod()-1
annual_return_bb.plot(kind='bar')
plt.title("Annual Returns for Bollinger Bands Mean Reversion")
plt.xlabel("Year")
plt.ylabel("Annual Return")
plt.show()

In [ ]:
# Total returns pivot table for Bollinger Bands strategy
total_return_bb = metrics_bb_by_year.pivot(index='Ticker', columns='Year', values='Total Return')
total_return_bb

In [ ]:

# BB strategy
bb_pivot = metrics_bb_by_year.groupby('Year')['Total Return'].mean().to_frame().T
bb_pivot.index = ['Bollinger Bands Strategy']
bb_pivot

In [ ]:
# 1. Calculate average annual return for your BB strategy
bb_annual_return = metrics_bb_by_year.groupby('Year')['Total Return'].mean()

In [ ]:
# 2. Calculate annual return for the mutual fund
fund_data['Fund Return'] = fund_data['Close'].pct_change()
fund_data = fund_data.dropna(subset=['Fund Return'])
fund_data['Year'] = fund_data.index.year
fund_annual_return = fund_data.groupby('Year')['Fund Return'].apply(lambda x: (1 + x).prod() - 1)

In [ ]:
# 3. Plot comparison
plt.figure(figsize=(12, 6))
plt.bar(bb_annual_return.index - 0.15, bb_annual_return.values, width=0.3, label='Bollinger Bands Strategy Avg Return', color='skyblue')
plt.bar(fund_annual_return.index + 0.15, fund_annual_return.values, width=0.3, label='Mutual Fund Return', color='orange')
plt.xlabel('Year')
plt.ylabel('Annual Return')
plt.title('Annual Return: Bollinger Bands Strategy (Avg) vs. Mutual Fund')
plt.legend()
plt.xticks(sorted(set(bb_annual_return.index) | set(fund_annual_return.index)), rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Summary table for Bollinger Bands strategy
bb_summary = pd.DataFrame({
    'Strategy': ['Bollinger Bands Mean Reversion'],
    'CAGR': [metrics_df_bb['CAGR'].mean()],
    'Total Return': [metrics_df_bb['Total Return'].mean()],
    'Volatility': [metrics_df_bb['Volatility'].mean()],
    'Max Drawdown': [metrics_df_bb['Max Drawdown'].mean()],
    'Sharpe Ratio': [metrics_df_bb['Sharpe Ratio'].mean()],
    'Win Rate (%)': [metrics_df_bb['Win Rate (%)'].mean()],
    'Profit Factor': [metrics_df_bb['Profit Factor'].mean()]
})
display(bb_summary)

In [ ]:
# Combined metrics plots for Bollinger Bands Mean Reversion Strategy

fig, axs = plt.subplots(3, 3, figsize=(18, 14))
fig.suptitle('Bollinger Bands Mean Reversion Strategy Metrics', fontsize=16)

# Cumulative Return by Ticker
axs[0, 0].bar(metrics_df_bb['Ticker'], metrics_df_bb['Cumulative Return'], color='mediumseagreen')
axs[0, 0].set_title('Cumulative Return')
axs[0, 0].set_xlabel('Ticker')
axs[0, 0].set_ylabel('Cumulative Return')
axs[0, 0].grid(axis='y', linestyle='--', alpha=0.7)

# CAGR by Ticker
axs[0, 1].bar(metrics_df_bb['Ticker'], metrics_df_bb['CAGR'], color='royalblue')
axs[0, 1].set_title('CAGR')
axs[0, 1].set_xlabel('Ticker')
axs[0, 1].set_ylabel('CAGR')
axs[0, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Total Return by Ticker
axs[0, 2].bar(metrics_df_bb['Ticker'], metrics_df_bb['Total Return'], color='seagreen')
axs[0, 2].set_title('Total Return')
axs[0, 2].set_xlabel('Ticker')
axs[0, 2].set_ylabel('Total Return')
axs[0, 2].grid(axis='y', linestyle='--', alpha=0.7)

#standard return by Ticker
axs[1, 0].bar(metrics_df_bb['Ticker'], metrics_df_bb['Standard Return'], color='slateblue')
axs[1, 0].set_title('Standard Return')
axs[1, 0].set_xlabel('Ticker')
axs[1, 0].set_ylabel('Standard Return')
axs[1, 0].grid(axis='y', linestyle='--', alpha=0.7)

#volatility by Ticker
axs[1, 1].bar(metrics_df_bb['Ticker'], metrics_df_bb['Volatility'], color='slateblue')
axs[1, 1].set_title('Volatility')
axs[1, 1].set_xlabel('Ticker')
axs[1, 1].set_ylabel('Volatility')
axs[1, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Sharpe Ratio by Ticker
axs[1, 2].bar(metrics_df_bb['Ticker'], metrics_df_bb['Sharpe Ratio'], color='orange')
axs[1, 2].set_title('Sharpe Ratio')
axs[1, 2].set_xlabel('Ticker')
axs[1, 2].set_ylabel('Sharpe Ratio')
axs[1, 2].grid(axis='y', linestyle='--', alpha=0.7)

# Max Drawdown by Ticker
axs[2, 0].bar(metrics_df_bb['Ticker'], metrics_df_bb['Max Drawdown'], color='crimson')
axs[2, 0].set_title('Max Drawdown')
axs[2, 0].set_xlabel('Ticker')
axs[2, 0].set_ylabel('Max Drawdown')
axs[2, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Win Rate by Ticker
axs[2, 1].bar(metrics_df_bb['Ticker'], metrics_df_bb['Win Rate (%)'], color='purple')
axs[2, 1].set_title('Win Rate (%)')
axs[2, 1].set_xlabel('Ticker')
axs[2, 1].set_ylabel('Win Rate (%)')
axs[2, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Profit Factor by Ticker
axs[2, 2].bar(metrics_df_bb['Ticker'], metrics_df_bb['Profit Factor'], color='teal')
axs[2, 2].set_title('Profit Factor')
axs[2, 2].set_xlabel('Ticker')
axs[2, 2].set_ylabel('Profit Factor')
axs[2, 2].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
# bollinger vs mutual fund summary table
bb_summary_table = pd.concat([bb_summary, fund_summary], ignore_index=True)
bb_summary_table.set_index('Strategy', inplace=True)
display(bb_summary_table)

**RSI Oversold/Overbought:**  
Computes 14-day RSI. Buys when RSI recovers from below 30, sells when RSI drops from above 70. Trades and metrics are calculated.


In [ ]:
# RSI generation: compute RSI, signals, pos markers and extract trades (compute step)
# Parameters
rsi_window = 14
rsi_oversold = 30
rsi_overbought = 70

rsi_all_trades = []
rsi_dict = {}

for ticker in holdings:
    print(f"Processing RSI for {ticker}...")
    try:
        stock_rsi = data[ticker][['Close']].dropna().copy()
        # Calculate RSI (Wilder's smoothing via ewm)
        delta = stock_rsi['Close'].diff()
        gain = delta.clip(lower=0)
        loss = -delta.clip(upper=0)
        avg_gain = gain.ewm(alpha=1/rsi_window, adjust=False).mean()
        avg_loss = loss.ewm(alpha=1/rsi_window, adjust=False).mean()
        rs = avg_gain / avg_loss.replace(0, np.nan)
        stock_rsi['RSI'] = 100 - (100 / (1 + rs))
        stock_rsi['RSI'] = stock_rsi['RSI'].fillna(50)

        # Signals: detect crossing back inside bands
        stock_rsi['Below_Oversold'] = stock_rsi['RSI'] < rsi_oversold
        stock_rsi['Above_Overbought'] = stock_rsi['RSI'] > rsi_overbought
        stock_rsi['Prev_Below_Oversold'] = stock_rsi['Below_Oversold'].shift(1).fillna(False)
        stock_rsi['Prev_Above_Overbought'] = stock_rsi['Above_Overbought'].shift(1).fillna(False)

        stock_rsi['Position'] = 0
        # Buy when RSI was oversold yesterday and now rises back to >= oversold (reversion)
        stock_rsi.loc[(stock_rsi['Prev_Below_Oversold'] == True) & (stock_rsi['RSI'] >= rsi_oversold), 'Position'] = 1
        # Sell when RSI was overbought yesterday and now falls to <= overbought (reversion)
        stock_rsi.loc[(stock_rsi['Prev_Above_Overbought'] == True) & (stock_rsi['RSI'] <= rsi_overbought), 'Position'] = -1

        # Optional holding logic (hold until opposite signal)
        stock_rsi['Hold_Position'] = 0
        current = 0
        for idx, row in stock_rsi.iterrows():
            if row['Position'] == 1:
                current = 1
            elif row['Position'] == -1:
                current = -1
            stock_rsi.at[idx, 'Hold_Position'] = current

        # Convert hold changes to Pos_Marker (entry/exit markers compatible with calculate_trades)
        stock_rsi['Prev_Hold'] = stock_rsi['Hold_Position'].shift(1).fillna(0)
        stock_rsi['Pos_Marker'] = 0
        stock_rsi.loc[(stock_rsi['Prev_Hold'] != 1) & (stock_rsi['Hold_Position'] == 1), 'Pos_Marker'] = 1
        stock_rsi.loc[(stock_rsi['Prev_Hold'] == 1) & (stock_rsi['Hold_Position'] != 1), 'Pos_Marker'] = -1

        # Extract trades using the existing calculate_trades function
        trades = calculate_trades(pd.DataFrame({'Close': stock_rsi['Close'], 'Position': stock_rsi['Pos_Marker']}, index=stock_rsi.index))
        if not trades.empty:
            trades['Ticker'] = ticker
            rsi_all_trades.append(trades)

        # store stock frame for plotting later
        rsi_dict[ticker] = stock_rsi[['Close', 'RSI', 'Position', 'Hold_Position', 'Pos_Marker']].copy()

    except Exception as e:
        print(f"Skipping {ticker}, error: {e}")
        continue

# Collect into DataFrame
if rsi_all_trades:
    rsi_all_trades_df = pd.concat(rsi_all_trades, ignore_index=True)
    rsi_all_trades_df['Sell Date'] = pd.to_datetime(rsi_all_trades_df['Sell Date'], format='%d-%m-%Y', errors='coerce')
    rsi_all_trades_df['Buy Date'] = pd.to_datetime(rsi_all_trades_df['Buy Date'], format='%d-%m-%Y', errors='coerce')
    rsi_all_trades_df['Year'] = rsi_all_trades_df['Sell Date'].dt.year
else:
    rsi_all_trades_df = pd.DataFrame()

print('RSI generation complete — rsi_all_trades_df and rsi_ma_dict are available. Run the RSI metrics cell next.')

In [ ]:
#group by ticker and year to calculate annual metrics for breakout strategy
metrics_rsi_by_year = rsi_all_trades_df.groupby(['Ticker', 'Year']).apply(calculate_metrics, risk_free_rate=0.02).reset_index(drop=True)


In [ ]:
#calculate overall metrics for each ticker for breakout strategy
metrics_rows_rsi = []
for ticker in holdings:
    ticker_trades_rsi = rsi_all_trades_df[rsi_all_trades_df['Ticker'] == ticker]
    metrics_row_rsi = calculate_metrics(ticker_trades_rsi)
    metrics_rows_rsi.append(metrics_row_rsi)
metrics_df_rsi = pd.concat(metrics_rows_rsi, ignore_index=True)

# 9. Display or plot results as needed
print(metrics_rsi_by_year.head())
print(metrics_df_rsi.head())

In [ ]:
# sorting values
metrics_rsi_by_year.sort_values(by=['Ticker', 'Year'], inplace=True)
metrics_rsi_by_year.head()

In [ ]:
# Plot RSI signals for each ticker
for ticker in rsi_dict:
    stock_rsi_plot = rsi_dict[ticker].copy()
    buy_signals = stock_rsi_plot[stock_rsi_plot['Pos_Marker'] == 1]
    sell_signals = stock_rsi_plot[stock_rsi_plot['Pos_Marker'] == -1]

    fig, axs = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    sns.set_style("whitegrid")

    # Price plot
    axs[0].plot(stock_rsi_plot.index, stock_rsi_plot['Close'], label='Close Price', color='blue', alpha=0.7)
    axs[0].scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy', s=80)
    axs[0].scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell', s=80)
    axs[0].set_ylabel("Price")
    axs[0].set_title(f"{ticker} Price & RSI Buy/Sell Signals")
    axs[0].legend()
    axs[0].grid(True, linestyle='--', alpha=0.5)

    # RSI plot
    axs[1].plot(stock_rsi_plot.index, stock_rsi_plot['RSI'], label='RSI', color='purple', alpha=0.7)
    axs[1].axhline(30, color='green', linestyle='--', label='Oversold (30)')
    axs[1].axhline(70, color='red', linestyle='--', label='Overbought (70)')
    axs[1].set_ylabel("RSI")
    axs[1].set_xlabel("Date")
    axs[1].legend()
    axs[1].grid(True, linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.show()

In [ ]:


tickers = list(rsi_dict.keys())
n_tickers = len(tickers)

sns.set_style("whitegrid")
fig, axes = plt.subplots(n_tickers * 2, 1, figsize=(18, 5 * n_tickers), sharex=False)
axes = axes.flatten()

for i, ticker in enumerate(tickers):
    stock = rsi_dict[ticker].copy()
    buy_signals = stock[stock['Pos_Marker'] == 1]
    sell_signals = stock[stock['Pos_Marker'] == -1]

    ax_top = axes[i * 2]
    ax_bottom = axes[i * 2 + 1]

    # Top: Close price + buy/sell signals
    ax_top.plot(stock.index, stock['Close'], label='Close Price', color='blue', alpha=0.7)
    ax_top.scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy', s=80)
    ax_top.scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell', s=80)
    ax_top.set_ylabel("Price")
    ax_top.set_title(f"{ticker} RSI Strategy Trades")
    ax_top.legend()
    ax_top.grid(True, linestyle='--', alpha=0.5)

    # Bottom: RSI values only
    ax_bottom.plot(stock.index, stock['RSI'], label='RSI', color='purple', alpha=0.7)
    ax_bottom.scatter(buy_signals.index, buy_signals['RSI'], marker='^', color='green', label='Buy', s=80)
    ax_bottom.scatter(sell_signals.index, sell_signals['RSI'], marker='v', color='red', label='Sell', s=80)
    ax_bottom.axhline(30, color='green', linestyle='--', label='Oversold (30)')
    ax_bottom.axhline(70, color='red', linestyle='--', label='Overbought (70)')
    ax_bottom.set_ylabel("RSI")
    ax_bottom.set_xlabel("Date")
    ax_bottom.set_title(f"{ticker} RSI (Oversold/Overbought)")
    ax_bottom.legend()
    ax_bottom.grid(True, linestyle='--', alpha=0.5)

plt.suptitle('RSI Oversold/Overbought Strategy - All Tickers', fontsize=18, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
#calculate overall metrics for each ticker 
metrics_rows_rsi = []
for ticker in holdings:
    ticker_trades_rsi = rsi_all_trades_df[rsi_all_trades_df['Ticker'] == ticker]
    metrics_row_rsi = calculate_metrics(ticker_trades_rsi)
    metrics_rows_rsi.append(metrics_row_rsi)
metrics_df_rsi = pd.concat(metrics_rows_rsi, ignore_index=True)

# 9. Display or plot results as needed
print(metrics_rsi_by_year.head())
print(metrics_df_rsi.head())


In [ ]:
# Total returns pivot table for RSI strategy
total_return_rsi = metrics_rsi_by_year.pivot(index='Ticker', columns='Year', values='Total Return')
total_return_rsi

In [ ]:
# RSI strategy
rsi_pivot = metrics_rsi_by_year.groupby('Year')['Total Return'].mean().to_frame().T
rsi_pivot.index = ['RSI Strategy']
rsi_pivot

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(18, 14))
fig.suptitle('RSI Mean Reversion Strategy Metrics', fontsize=16)

# Cumulative Return by Ticker
axs[0, 0].bar(metrics_df_rsi['Ticker'], metrics_df_rsi['Cumulative Return'], color='mediumseagreen')
axs[0, 0].set_title('Cumulative Return')
axs[0, 0].set_xlabel('Ticker')
axs[0, 0].set_ylabel('Cumulative Return')
axs[0, 0].grid(axis='y', linestyle='--', alpha=0.7)

# CAGR by Ticker
axs[0, 1].bar(metrics_df_rsi['Ticker'], metrics_df_rsi['CAGR'], color='royalblue')
axs[0, 1].set_title('CAGR')
axs[0, 1].set_xlabel('Ticker')
axs[0, 1].set_ylabel('CAGR')
axs[0, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Total Return by Ticker
axs[0, 2].bar(metrics_df_rsi['Ticker'], metrics_df_rsi['Total Return'], color='seagreen')
axs[0, 2].set_title('Total Return')
axs[0, 2].set_xlabel('Ticker')
axs[0, 2].set_ylabel('Total Return')
axs[0, 2].grid(axis='y', linestyle='--', alpha=0.7)

# Standard Return by Ticker
axs[1, 0].bar(metrics_df_rsi['Ticker'], metrics_df_rsi['Standard Return'], color='slateblue')
axs[1, 0].set_title('Standard Return')
axs[1, 0].set_xlabel('Ticker')
axs[1, 0].set_ylabel('Standard Return')
axs[1, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Volatility by Ticker
axs[1, 1].bar(metrics_df_rsi['Ticker'], metrics_df_rsi['Volatility'], color='slateblue')
axs[1, 1].set_title('Volatility')
axs[1, 1].set_xlabel('Ticker')
axs[1, 1].set_ylabel('Volatility')
axs[1, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Sharpe Ratio by Ticker
axs[1, 2].bar(metrics_df_rsi['Ticker'], metrics_df_rsi['Sharpe Ratio'], color='orange')
axs[1, 2].set_title('Sharpe Ratio')
axs[1, 2].set_xlabel('Ticker')
axs[1, 2].set_ylabel('Sharpe Ratio')
axs[1, 2].grid(axis='y', linestyle='--', alpha=0.7)

# Max Drawdown by Ticker
axs[2, 0].bar(metrics_df_rsi['Ticker'], metrics_df_rsi['Max Drawdown'], color='crimson')
axs[2, 0].set_title('Max Drawdown')
axs[2, 0].set_xlabel('Ticker')
axs[2, 0].set_ylabel('Max Drawdown')
axs[2, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Win Rate (%) by Ticker
axs[2, 1].bar(metrics_df_rsi['Ticker'], metrics_df_rsi['Win Rate (%)'], color='purple')
axs[2, 1].set_title('Win Rate (%)')
axs[2, 1].set_xlabel('Ticker')
axs[2, 1].set_ylabel('Win Rate (%)')
axs[2, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Profit Factor by Ticker
axs[2, 2].bar(metrics_df_rsi['Ticker'], metrics_df_rsi['Profit Factor'], color='teal')
axs[2, 2].set_title('Profit Factor')
axs[2, 2].set_xlabel('Ticker')
axs[2, 2].set_ylabel('Profit Factor')
axs[2, 2].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
# 1. Calculate average annual return for your RSI strategy
rsi_annual_return = metrics_rsi_by_year.groupby('Year')['Total Return'].mean()

In [ ]:
# 2. Calculate annual return for the mutual fund
fund_data['Fund Return'] = fund_data['Close'].pct_change()
fund_data = fund_data.dropna(subset=['Fund Return'])
fund_data['Year'] = fund_data.index.year
fund_annual_return = fund_data.groupby('Year')['Fund Return'].apply(lambda x: (1 + x).prod() - 1)

In [ ]:
# 3. Plot comparison
plt.figure(figsize=(12, 6))
plt.bar(rsi_annual_return.index - 0.15, rsi_annual_return.values, width=0.3, label='RSI Strategy Avg Return', color='skyblue')
plt.bar(fund_annual_return.index + 0.15, fund_annual_return.values, width=0.3, label='Mutual Fund Return', color='orange')
plt.xlabel('Year')
plt.ylabel('Annual Return')
plt.title('Annual Return: RSI Strategy (Avg) vs. Mutual Fund')
plt.legend()
plt.xticks(sorted(set(rsi_annual_return.index) | set(fund_annual_return.index)), rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Summary table for Bollinger Bands strategy
rsi_summary = pd.DataFrame({
    'Strategy': ['RSI Oversold/Overbought'],
    'CAGR': [metrics_df_rsi['CAGR'].mean()],
    'Total Return': [metrics_df_rsi['Total Return'].mean()],
    'Volatility': [metrics_df_rsi['Volatility'].mean()],
    'Max Drawdown': [metrics_df_rsi['Max Drawdown'].mean()],
    'Sharpe Ratio': [metrics_df_rsi['Sharpe Ratio'].mean()],
    'Win Rate (%)': [metrics_df_rsi['Win Rate (%)'].mean()],
    'Profit Factor': [metrics_df_rsi['Profit Factor'].mean()]
})
display(rsi_summary)

In [ ]:
# rsi vs mutual fund summary table
rsi_summary_table = pd.concat([rsi_summary, fund_summary], ignore_index=True)
rsi_summary_table.set_index('Strategy', inplace=True)
display(rsi_summary_table)

**MA Envelope Reversal:**  
Calculates a moving average and envelopes (±2%). Buys when price reverts above the lower envelope, sells when price reverts below the upper envelope. Trades and metrics are calculated.

In [ ]:
# --- Moving Average Envelope Reversal: compute MA, envelopes, signals, pos markers and extract trades ---

# Parameters for MA Envelope
env_window = 20          # length of moving average
env_pct    = 0.02        # 2% envelope width (±2%)

env_all_trades = []
env_dict = {}

for ticker in holdings:
    print(f"Processing MA Envelope Reversal for {ticker}...")
    try:
        # base price series
        stock_env = data[ticker][['Close']].dropna().copy()

        # 1) Compute moving average and envelope bands
        stock_env['MA']       = stock_env['Close'].rolling(window=env_window, min_periods=env_window).mean()
        stock_env['Env_Up']   = stock_env['MA'] * (1 + env_pct)
        stock_env['Env_Down'] = stock_env['MA'] * (1 - env_pct)

        # Drop early rows where MA is NaN
        stock_env = stock_env.dropna(subset=['MA'])

        # 2) Define conditions: price below lower band / above upper band
        stock_env['Below_Lower']  = stock_env['Close'] < stock_env['Env_Down']
        stock_env['Above_Upper']  = stock_env['Close'] > stock_env['Env_Up']
        stock_env['Prev_Below_Lower'] = stock_env['Below_Lower'].shift(1).fillna(False)
        stock_env['Prev_Above_Upper'] = stock_env['Above_Upper'].shift(1).fillna(False)

        # 3) Reversal signals
        stock_env['Position'] = 0

        # Buy when price was below lower band yesterday and comes back inside today
        stock_env.loc[
            (stock_env['Prev_Below_Lower'] == True) & (stock_env['Close'] >= stock_env['Env_Down']),
            'Position'
        ] = 1

        # Sell when price was above upper band yesterday and comes back inside today
        stock_env.loc[
            (stock_env['Prev_Above_Upper'] == True) & (stock_env['Close'] <= stock_env['Env_Up']),
            'Position'
        ] = -1

        # 4) Optional holding logic: stay long until an exit signal
        stock_env['Hold_Position'] = 0
        current = 0
        for idx, row in stock_env.iterrows():
            if row['Position'] == 1:
                current = 1          # enter / stay long
            elif row['Position'] == -1:
                current = 0          # exit to flat on sell signal
            stock_env.at[idx, 'Hold_Position'] = current

        # 5) Convert hold changes into Pos_Marker for calculate_trades
        stock_env['Prev_Hold'] = stock_env['Hold_Position'].shift(1).fillna(0)
        stock_env['Pos_Marker'] = 0
        # Entry: go from 0 to 1
        stock_env.loc[
            (stock_env['Prev_Hold'] != 1) & (stock_env['Hold_Position'] == 1),
            'Pos_Marker'
        ] = 1
        # Exit: go from 1 to 0
        stock_env.loc[
            (stock_env['Prev_Hold'] == 1) & (stock_env['Hold_Position'] != 1),
            'Pos_Marker'
        ] = -1

        # 6) Extract trades using existing calculate_trades
        trades = calculate_trades(
            pd.DataFrame(
                {'Close': stock_env['Close'], 'Position': stock_env['Pos_Marker']},
                index=stock_env.index
            )
        )
        if not trades.empty:
            trades['Ticker'] = ticker
            env_all_trades.append(trades)

        # 7) Store full frame for plotting later
        env_dict[ticker] = stock_env[[
            'Close', 'MA', 'Env_Up', 'Env_Down',
            'Position', 'Hold_Position', 'Pos_Marker'
        ]].copy()

    except Exception as e:
        print(f"Skipping {ticker}, error: {e}")
        continue

# 8) Collect all trades into a DataFrame
if env_all_trades:
    env_all_trades_df = pd.concat(env_all_trades, ignore_index=True)
    env_all_trades_df['Sell Date'] = pd.to_datetime(env_all_trades_df['Sell Date'],
                                                    format='%d-%m-%Y', errors='coerce')
    env_all_trades_df['Buy Date']  = pd.to_datetime(env_all_trades_df['Buy Date'],
                                                    format='%d-%m-%Y', errors='coerce')
    env_all_trades_df['Year'] = env_all_trades_df['Sell Date'].dt.year
else:
    env_all_trades_df = pd.DataFrame()

print("MA Envelope Reversal generation complete — env_all_trades_df and env_ma_dict are available. Run the Envelope metrics cell next.")


In [ ]:
#group by ticker and year to calculate annual metrics for breakout strategy
metrics_env_by_year = env_all_trades_df.groupby(['Ticker', 'Year']).apply(calculate_metrics, risk_free_rate=0.02).reset_index(drop=True)


In [ ]:
#calculate overall metrics for each ticker for breakout strategy
metrics_rows_env = []
for ticker in holdings:
    ticker_trades_env = env_all_trades_df[env_all_trades_df['Ticker'] == ticker]
    metrics_row_env = calculate_metrics(ticker_trades_env)
    metrics_rows_env.append(metrics_row_env)
metrics_df_env = pd.concat(metrics_rows_env, ignore_index=True)

# 9. Display or plot results as needed
print(metrics_env_by_year.head())
print(metrics_df_env.head())

In [ ]:
# sorting values
metrics_env_by_year.sort_values(by=['Ticker', 'Year'], inplace=True)
metrics_env_by_year.head()

In [ ]:
# Plot MA Envelope signals for each ticker
for ticker in env_dict:
    stock_env_plot = env_dict[ticker].copy()
    buy_signals = stock_env_plot[stock_env_plot['Pos_Marker'] == 1]
    sell_signals = stock_env_plot[stock_env_plot['Pos_Marker'] == -1]

    fig, axs = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    sns.set_style("whitegrid")

    # Top subplot: Close price + buy/sell signals
    axs[0].plot(stock_env_plot.index, stock_env_plot['Close'], label='Close Price', color='blue', alpha=0.7)
    axs[0].scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy', s=80)
    axs[0].scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell', s=80)
    axs[0].set_ylabel("Price")
    axs[0].set_title(f"{ticker} Close Price & Buy/Sell Signals (MA Envelope)")
    axs[0].legend()
    axs[0].grid(True, linestyle='--', alpha=0.5)

    # Bottom subplot: MA Envelope bands
    axs[1].plot(stock_env_plot.index, stock_env_plot['MA'], label='MA', color='orange')
    axs[1].plot(stock_env_plot.index, stock_env_plot['Env_Up'], label='Envelope Up', color='green', linestyle='--')
    axs[1].plot(stock_env_plot.index, stock_env_plot['Env_Down'], label='Envelope Down', color='red', linestyle='--')
    axs[1].set_ylabel("Envelope Value")
    axs[1].set_xlabel("Date")
    axs[1].set_title(f"{ticker} MA Envelope Bands")
    axs[1].legend()
    axs[1].grid(True, linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.show()

In [ ]:
tickers = list(env_dict.keys())
n_tickers = len(tickers)

sns.set_style("whitegrid")
fig, axes = plt.subplots(n_tickers * 2, 1, figsize=(18, 5 * n_tickers), sharex=False)
axes = axes.flatten()

for i, ticker in enumerate(tickers):
    stock = env_dict[ticker].copy()
    buy_signals = stock[stock['Pos_Marker'] == 1]
    sell_signals = stock[stock['Pos_Marker'] == -1]

    ax_top = axes[i * 2]
    ax_bottom = axes[i * 2 + 1]

    # Top: Close price + buy/sell signals
    ax_top.plot(stock.index, stock['Close'], label='Close Price', color='blue', alpha=0.7)
    ax_top.scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy', s=80)
    ax_top.scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell', s=80)
    ax_top.set_ylabel("Price")
    ax_top.set_title(f"{ticker} MA Envelope Strategy Trades")
    ax_top.legend()
    ax_top.grid(True, linestyle='--', alpha=0.5)

    # Bottom: Envelope bands only
    ax_bottom.plot(stock.index, stock['MA'], label='MA', color='orange')
    ax_bottom.plot(stock.index, stock['Env_Up'], label='Envelope Up', color='green', linestyle='--')
    ax_bottom.plot(stock.index, stock['Env_Down'], label='Envelope Down', color='red', linestyle='--')
    ax_bottom.set_ylabel("Envelope Value")
    ax_bottom.set_xlabel("Date")
    ax_bottom.set_title(f"{ticker} MA Envelope Bands")
    ax_bottom.legend()
    ax_bottom.grid(True, linestyle='--', alpha=0.5)

plt.suptitle('MA Envelope Reversal Strategy - All Tickers', fontsize=18, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
#calculate overall metrics for each ticker for breakout strategy
metrics_rows_env = []
for ticker in holdings:
    ticker_trades_env = env_all_trades_df[env_all_trades_df['Ticker'] == ticker]
    metrics_row_env = calculate_metrics(ticker_trades_env)
    metrics_rows_env.append(metrics_row_env)
metrics_df_env = pd.concat(metrics_rows_env, ignore_index=True)

# 9. Display or plot results as needed
print(metrics_env_by_year.head())
print(metrics_df_env.head())

In [ ]:
# total returns pivot table for MA Envelope strategy
total_return_env = metrics_env_by_year.pivot(index='Ticker', columns='Year', values='Total Return')
total_return_env

In [ ]:
# ENV strategy
env_pivot = metrics_env_by_year.groupby('Year')['Total Return'].mean().to_frame().T
env_pivot.index = ['ENV Strategy']
env_pivot

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(18, 14))
fig.suptitle('MA Envelope Reversal Strategy Metrics', fontsize=16)

# Cumulative Return by Ticker
axs[0, 0].bar(metrics_df_env['Ticker'], metrics_df_env['Cumulative Return'], color='mediumseagreen')
axs[0, 0].set_title('Cumulative Return')
axs[0, 0].set_xlabel('Ticker')
axs[0, 0].set_ylabel('Cumulative Return')
axs[0, 0].grid(axis='y', linestyle='--', alpha=0.7)

# CAGR by Ticker
axs[0, 1].bar(metrics_df_env['Ticker'], metrics_df_env['CAGR'], color='royalblue')
axs[0, 1].set_title('CAGR')
axs[0, 1].set_xlabel('Ticker')
axs[0, 1].set_ylabel('CAGR')
axs[0, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Total Return by Ticker
axs[0, 2].bar(metrics_df_env['Ticker'], metrics_df_env['Total Return'], color='seagreen')
axs[0, 2].set_title('Total Return')
axs[0, 2].set_xlabel('Ticker')
axs[0, 2].set_ylabel('Total Return')
axs[0, 2].grid(axis='y', linestyle='--', alpha=0.7)

# Standard Return by Ticker
axs[1, 0].bar(metrics_df_env['Ticker'], metrics_df_env['Standard Return'], color='slateblue')
axs[1, 0].set_title('Standard Return')
axs[1, 0].set_xlabel('Ticker')
axs[1, 0].set_ylabel('Standard Return')
axs[1, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Volatility by Ticker
axs[1, 1].bar(metrics_df_env['Ticker'], metrics_df_env['Volatility'], color='slateblue')
axs[1, 1].set_title('Volatility')
axs[1, 1].set_xlabel('Ticker')
axs[1, 1].set_ylabel('Volatility')
axs[1, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Sharpe Ratio by Ticker
axs[1, 2].bar(metrics_df_env['Ticker'], metrics_df_env['Sharpe Ratio'], color='orange')
axs[1, 2].set_title('Sharpe Ratio')
axs[1, 2].set_xlabel('Ticker')
axs[1, 2].set_ylabel('Sharpe Ratio')
axs[1, 2].grid(axis='y', linestyle='--', alpha=0.7)

# Max Drawdown by Ticker
axs[2, 0].bar(metrics_df_env['Ticker'], metrics_df_env['Max Drawdown'], color='crimson')
axs[2, 0].set_title('Max Drawdown')
axs[2, 0].set_xlabel('Ticker')
axs[2, 0].set_ylabel('Max Drawdown')
axs[2, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Win Rate (%) by Ticker
axs[2, 1].bar(metrics_df_env['Ticker'], metrics_df_env['Win Rate (%)'], color='purple')
axs[2, 1].set_title('Win Rate (%)')
axs[2, 1].set_xlabel('Ticker')
axs[2, 1].set_ylabel('Win Rate (%)')
axs[2, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Profit Factor by Ticker
axs[2, 2].bar(metrics_df_env['Ticker'], metrics_df_env['Profit Factor'], color='teal')
axs[2, 2].set_title('Profit Factor')
axs[2, 2].set_xlabel('Ticker')
axs[2, 2].set_ylabel('Profit Factor')
axs[2, 2].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
# 1. Calculate average annual return for your MA strategy
env_annual_return = metrics_env_by_year.groupby('Year')['Total Return'].mean()

In [ ]:
# 2. Calculate annual return for the mutual fund
fund_data['Fund Return'] = fund_data['Close'].pct_change()
fund_data = fund_data.dropna(subset=['Fund Return'])
fund_data['Year'] = fund_data.index.year
fund_annual_return = fund_data.groupby('Year')['Fund Return'].apply(lambda x: (1 + x).prod() - 1)

In [ ]:
# 3. Plot comparison
plt.figure(figsize=(12, 6))
plt.bar(env_annual_return.index - 0.15, env_annual_return.values, width=0.3, label='MA Envelope Strategy Avg Return', color='skyblue')
plt.bar(fund_annual_return.index + 0.15, fund_annual_return.values, width=0.3, label='Mutual Fund Return', color='orange')
plt.xlabel('Year')
plt.ylabel('Annual Return')
plt.title('Annual Return: MA Envelope Strategy (Avg) vs. Mutual Fund')
plt.legend()
plt.xticks(sorted(set(env_annual_return.index) | set(fund_annual_return.index)), rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:

# Summary table for MA Envelope Reversal Strategy Bands strategy
env_summary = pd.DataFrame({
    'Strategy': ['MA Envelope Reversal Strategy Bands'],
    'CAGR': [metrics_df_env['CAGR'].mean()],
    'Total Return': [metrics_df_env['Total Return'].mean()],
    'Volatility': [metrics_df_env['Volatility'].mean()],
    'Max Drawdown': [metrics_df_env['Max Drawdown'].mean()],
    'Sharpe Ratio': [metrics_df_env['Sharpe Ratio'].mean()],
    'Win Rate (%)': [metrics_df_env['Win Rate (%)'].mean()],
    'Profit Factor': [metrics_df_env['Profit Factor'].mean()]
})
display(env_summary)

In [ ]:
# MA Envelope Reversal Strategy Bands vs mutual fund summary table
env_summary_table = pd.concat([env_summary, fund_summary], ignore_index=True)
env_summary_table.set_index('Strategy', inplace=True)
display(env_summary_table)

In [ ]:
#combine strategy metrics for comparison
meanrev_startegies_summary = pd.concat([bb_summary, rsi_summary, env_summary], ignore_index=True)
meanrev_startegies_summary.set_index('Strategy', inplace=True)
display(meanrev_startegies_summary)

In [ ]:
# Combine all strategy pivots into one DataFrame
combined_pivot_meanrev = pd.concat([bb_pivot, rsi_pivot, env_pivot], axis=0)
display(combined_pivot_meanrev)

# MOMEMTUM STRATEGY

##### 1) Rate of Change Strategy

In [ ]:
#calculate rate of change and trades for each stock
roc_all_trades = []
roc_dict = {}

for ticker in holdings:
    print(f"Processing {ticker}...")
    try:
        stock_roc = data[ticker][['Close']].dropna().copy()
        stock_roc['ROC'] = stock_roc['Close'].pct_change(periods=5) * 100
        roc_dict[ticker] = stock_roc['ROC']
        stock_roc.loc[stock_roc.index[5:], 'Position'] = np.where(stock_roc['ROC'][5:] > 0, 1, -1)

        roc_dict[ticker] = stock_roc[['Close', 'ROC','Position']].copy()
        trades = calculate_trades(stock_roc)
        if not trades.empty:
            trades['Ticker'] = ticker
            roc_all_trades.append(trades)
    except KeyError as e:
        print(f"Error processing {ticker}: {e}")
        continue
roc_all_trades_df = pd.concat(roc_all_trades,ignore_index = True)
if not roc_all_trades_df.empty:
    columns = ['Ticker'] + [col for col in roc_all_trades_df.columns if col != 'Ticker']
    roc_all_trades_df = roc_all_trades_df[columns]

In [ ]:
#Ensure "Sell Date" is datetime and extract year
roc_all_trades_df['Sell Date'] = pd.to_datetime(roc_all_trades_df['Sell Date'], format='%d-%m-%Y', errors='coerce')
roc_all_trades_df['Buy Date'] = pd.to_datetime(roc_all_trades_df['Buy Date'], format='%d-%m-%Y', errors='coerce')
roc_all_trades_df['Year'] = roc_all_trades_df['Sell Date'].dt.year

In [ ]:
#group by ticker and year to calculate annual metrics for each group
roc_metrics_by_year = roc_all_trades_df.groupby(['Ticker', 'Year']).apply(calculate_metrics, risk_free_rate=0.02).reset_index(drop=True)


In [ ]:
#Calculate overall metrics for each ticker
metrics_rows = []
for ticker in holdings:
    ticker_trades = roc_all_trades_df[roc_all_trades_df['Ticker'] == ticker]
    metrics_row = calculate_metrics(ticker_trades)
    metrics_rows.append(metrics_row)
roc_metrics_df = pd.concat(metrics_rows, ignore_index=True)

# 9. Display or plot results as needed
print(roc_metrics_by_year.head())
print(roc_metrics_df.head())

In [ ]:
# Plot ROC signals for each ticker
for ticker in roc_dict:
    stock_roc_plot = roc_dict[ticker].copy()
    # Identify buy/sell signals (where Position == 1 or -1)
    buy_signals = stock_roc_plot[stock_roc_plot['Position'] == 1]
    sell_signals = stock_roc_plot[stock_roc_plot['Position'] == -1]

    fig, axs = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    sns.set_style("whitegrid")

    # Top subplot: Close price + buy/sell signals
    axs[0].plot(stock_roc_plot.index, stock_roc_plot['Close'], label='Close Price', color='blue', alpha=0.7)
    axs[0].scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy', s=80)
    axs[0].scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell', s=80)
    axs[0].set_ylabel("Price")
    axs[0].set_title(f"{ticker} Close Price & ROC Buy/Sell Signals")
    axs[0].legend()
    axs[0].grid(True, linestyle='--', alpha=0.5)

    # Bottom subplot: ROC values
    axs[1].plot(stock_roc_plot.index, stock_roc_plot['ROC'], label='ROC (5-day %)', color='purple')
    axs[1].set_ylabel("ROC (%)")
    axs[1].set_xlabel("Date")
    axs[1].set_title(f"{ticker} Rate of Change (ROC)")
    axs[1].legend()
    axs[1].grid(True, linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.show()

The ROC strategy plot shows a very large number of buy and sell markers because the Rate of Change indicator is highly sensitive to small day-to-day price movements. Since ROC frequently crosses above and below the threshold levels, the strategy generates continuous signals whenever momentum shifts even slightly. Without smoothing or wider thresholds, this results in many rapid buy/sell triggers and a visually cluttered chart, reflecting the noisy nature of raw ROC-based trading

In [ ]:
tickers = list(roc_dict.keys())
n_tickers = len(tickers)

sns.set_style("whitegrid")
fig, axes = plt.subplots(n_tickers * 2, 1, figsize=(18, 5 * n_tickers), sharex=False)
axes = axes.flatten()

for i, ticker in enumerate(tickers):
    stock = roc_dict[ticker].copy()
    buy_signals = stock[stock['Position'] == 1]
    sell_signals = stock[stock['Position'] == -1]

    ax_top = axes[i * 2]
    ax_bottom = axes[i * 2 + 1]

    # Top: Close price + buy/sell signals
    ax_top.plot(stock.index, stock['Close'], label='Close Price', color='blue', alpha=0.7)
    ax_top.scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy', s=80)
    ax_top.scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell', s=80)
    ax_top.set_ylabel("Price")
    ax_top.set_title(f"{ticker} ROC Strategy Trades")
    ax_top.legend()
    ax_top.grid(True, linestyle='--', alpha=0.5)

    # Bottom: ROC values only
    ax_bottom.plot(stock.index, stock['ROC'], label='ROC (5-day %)', color='purple')
    ax_bottom.set_ylabel("ROC (%)")
    ax_bottom.set_xlabel("Date")
    ax_bottom.set_title(f"{ticker} Rate of Change (ROC)")
    ax_bottom.legend()
    ax_bottom.grid(True, linestyle='--', alpha=0.5)

plt.suptitle('Rate of Change (ROC) Strategy - All Tickers', fontsize=18, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# sorting values
roc_metrics_by_year.sort_values(by=['Ticker', 'Year'], inplace=True)
roc_metrics_by_year.head()

In [ ]:
# total returns pivot table for ROC strategy
total_return_roc = roc_metrics_by_year.pivot(index='Ticker', columns='Year', values='Total Return')
total_return_roc

In [ ]:
# ROC strategy
roc_pivot = roc_metrics_by_year.groupby('Year')['Total Return'].mean().to_frame().T
roc_pivot.index = ['ROC Strategy']
roc_pivot

In [ ]:
#plot for overall metrics
fig, axs = plt.subplots(3, 3, figsize=(18, 14))
fig.suptitle('ROC Strategy Metrics', fontsize=16)

# Cumulative Return by Ticker
axs[0, 0].bar(roc_metrics_df['Ticker'], roc_metrics_df['Cumulative Return'], color='mediumseagreen')
axs[0, 0].set_title('Cumulative Return')
axs[0, 0].set_xlabel('Ticker')
axs[0, 0].set_ylabel('Cumulative Return')
axs[0, 0].grid(axis='y', linestyle='--', alpha=0.7)

# CAGR by Ticker
axs[0, 1].bar(roc_metrics_df['Ticker'], roc_metrics_df['CAGR'], color='royalblue')
axs[0, 1].set_title('CAGR')
axs[0, 1].set_xlabel('Ticker')
axs[0, 1].set_ylabel('CAGR')
axs[0, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Total Return by Ticker
axs[0, 2].bar(roc_metrics_df['Ticker'], roc_metrics_df['Total Return'], color='seagreen')
axs[0, 2].set_title('Total Return')
axs[0, 2].set_xlabel('Ticker')
axs[0, 2].set_ylabel('Total Return')
axs[0, 2].grid(axis='y', linestyle='--', alpha=0.7)

# Standard Return by Ticker
axs[1, 0].bar(roc_metrics_df['Ticker'], roc_metrics_df['Standard Return'], color='slateblue')
axs[1, 0].set_title('Standard Return')
axs[1, 0].set_xlabel('Ticker')
axs[1, 0].set_ylabel('Standard Return')
axs[1, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Volatility by Ticker
axs[1, 1].bar(roc_metrics_df['Ticker'], roc_metrics_df['Volatility'], color='slateblue')
axs[1, 1].set_title('Volatility')       
axs[1, 1].set_xlabel('Ticker')
axs[1, 1].set_ylabel('Volatility')
axs[1, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Sharpe Ratio by Ticker
axs[1, 2].bar(roc_metrics_df['Ticker'], roc_metrics_df['Sharpe Ratio'], color='orange')
axs[1, 2].set_title('Sharpe Ratio')
axs[1, 2].set_xlabel('Ticker')
axs[1, 2].set_ylabel('Sharpe Ratio')
axs[1, 2].grid(axis='y', linestyle='--', alpha=0.7)

# Max Drawdown by Ticker
axs[2, 0].bar(roc_metrics_df['Ticker'], roc_metrics_df['Max Drawdown'], color='crimson')
axs[2, 0].set_title('Max Drawdown')
axs[2, 0].set_xlabel('Ticker')
axs[2, 0].set_ylabel('Max Drawdown')
axs[2, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Win Rate (%) by Ticker
axs[2, 1].bar(roc_metrics_df['Ticker'], roc_metrics_df['Win Rate (%)'], color='purple')
axs[2, 1].set_title('Win Rate (%)')
axs[2, 1].set_xlabel('Ticker')
axs[2, 1].set_ylabel('Win Rate (%)')
axs[2, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Profit Factor by Ticker
axs[2, 2].bar(roc_metrics_df['Ticker'], roc_metrics_df['Profit Factor'], color='teal')
axs[2, 2].set_title('Profit Factor')
axs[2, 2].set_xlabel('Ticker')
axs[2, 2].set_ylabel('Profit Factor')
axs[2, 2].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:

# 1. Calculate average annual return for your MA strategy
roc_annual_return = roc_metrics_by_year.groupby('Year')['Total Return'].mean()

In [ ]:
# 2. Calculate annual return for the mutual fund
fund_data['Fund Return'] = fund_data['Close'].pct_change()
fund_data = fund_data.dropna(subset=['Fund Return'])
fund_data['Year'] = fund_data.index.year
fund_annual_return = fund_data.groupby('Year')['Fund Return'].apply(lambda x: (1 + x).prod() - 1)

In [ ]:
# 3. Plot comparison
plt.figure(figsize=(12, 6))
plt.bar(roc_annual_return.index - 0.15, roc_annual_return.values, width=0.3, label='ROC Strategy Avg Return', color='skyblue')
plt.bar(fund_annual_return.index + 0.15, fund_annual_return.values, width=0.3, label='Mutual Fund Return', color='orange')
plt.xlabel('Year')
plt.ylabel('Annual Return')
plt.title('Annual Return: ROC Strategy (Avg) vs. Mutual Fund')
plt.legend()
plt.xticks(sorted(set(roc_annual_return.index) | set(fund_annual_return.index)), rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# --- Summary Table: ROC Strategy (Avg) vs. Mutual Fund ---

# 1. ROC Strategy (average across tickers)
roc_summary = pd.DataFrame({
    'Strategy': ['ROC Strategy (Avg)'],
    'CAGR': [roc_metrics_df['CAGR'].mean()],
    'Total Return': [roc_metrics_df['Total Return'].mean()],
    'Volatility': [roc_metrics_df['Volatility'].mean()],
    'Max Drawdown': [roc_metrics_df['Max Drawdown'].mean()],
    'Sharpe Ratio': [roc_metrics_df['Sharpe Ratio'].mean()],
    'Win Rate (%)': [roc_metrics_df['Win Rate (%)'].mean()],
    'Profit Factor': [roc_metrics_df['Profit Factor'].mean()]
})

In [ ]:
# roc vs mutual fund summary table
roc_summary_table = pd.concat([roc_summary, fund_summary], ignore_index=True)
roc_summary_table.set_index('Strategy', inplace=True)
display(roc_summary_table)

**52-Week High/Low Breakout Momentum:**  
Finds 52-week high/low. Buys when price breaks above the high, sells when it breaks below the low. Trades and metrics are calculated.

In [ ]:
# Calculate 52-Week High/Low Breakout Momentum Strategy
ftw_all_trades = []
ftw_dict = {}

for ticker in holdings:
    print(f"Processing {ticker} for 52-week high/low breakout...")
    try:
        stock_ftw = data[ticker][['Close']].dropna().copy()
        stock_ftw['52w_high'] = stock_ftw['Close'].rolling(window=252).max()
        stock_ftw['52w_low'] = stock_ftw['Close'].rolling(window=252).min()
        # Buy if price breaks above 52-week high, Sell if price breaks below 52-week low
        stock_ftw['Position'] = 0
        stock_ftw.loc[stock_ftw['Close'] >= stock_ftw['52w_high'], 'Position'] = 1
        stock_ftw.loc[stock_ftw['Close'] <= stock_ftw['52w_low'], 'Position'] = -1
        ftw_dict[ticker] = stock_ftw[['Close', '52w_high', '52w_low','Position']].copy()
        trades = calculate_trades(stock_ftw)
        if not trades.empty:
            trades['Ticker'] = ticker
            ftw_all_trades.append(trades)
    except KeyError as e:
        print(f"Error processing {ticker}: {e}")
        continue

if ftw_all_trades:
    ftw_all_trades_df = pd.concat(ftw_all_trades, ignore_index=True)
    # Convert dates and add year column
    if 'Sell Date' in ftw_all_trades_df.columns:
        ftw_all_trades_df['Sell Date'] = pd.to_datetime(ftw_all_trades_df['Sell Date'], format='%d-%m-%Y', errors='coerce')
        ftw_all_trades_df['Buy Date'] = pd.to_datetime(ftw_all_trades_df['Buy Date'], format='%d-%m-%Y', errors='coerce')
        ftw_all_trades_df['Year'] = ftw_all_trades_df['Sell Date'].dt.year
else:
    ftw_all_trades_df = pd.DataFrame()  # Empty DataFrame if no trades

ftw_all_trades_df.head()

In [ ]:
#group by ticker and year to calculate annual metrics for each group
ftw_metrics_by_year = ftw_all_trades_df.groupby(['Ticker', 'Year']).apply(calculate_metrics, risk_free_rate=0.02).reset_index(drop=True)

#Calculate overall metrics for each ticker
metrics_rows = []
for ticker in holdings:
    ticker_trades = ftw_all_trades_df[ftw_all_trades_df['Ticker'] == ticker]
    metrics_row = calculate_metrics(ticker_trades)
    metrics_rows.append(metrics_row)
ftw_metrics_df = pd.concat(metrics_rows, ignore_index=True)

# 9. Display or plot results as needed
print(ftw_metrics_by_year.head())

In [ ]:
# Total returns pivot table for 52-week high/low breakout strategy
total_return_ftw = ftw_metrics_by_year.pivot(index='Ticker', columns='Year', values='Total Return')
total_return_ftw

In [ ]:
# FTW strategy
ftw_pivot = ftw_metrics_by_year.groupby('Year')['Total Return'].mean().to_frame().T
ftw_pivot.index = ['FTW Strategy']
ftw_pivot

In [ ]:
# Plot 52-week high/low breakout signals for each ticker
for ticker in ftw_dict:
    stock_ftw_plot = ftw_dict[ticker].copy()
    # If you want buy/sell markers, you need Position column in ftw_dict
    # Add this if not present:
    if 'Position' not in stock_ftw_plot.columns and ticker in ftw_all_trades_df['Ticker'].unique():
        # Recreate Position from trades if needed
        stock_ftw_plot = stock_ftw_plot.join(
            ftw_all_trades_df[ftw_all_trades_df['Ticker'] == ticker][['Buy Date', 'Sell Date']],
            how='left'
        )
    buy_signals = stock_ftw_plot[stock_ftw_plot.get('Position', pd.Series([0]*len(stock_ftw_plot))) == 1]
    sell_signals = stock_ftw_plot[stock_ftw_plot.get('Position', pd.Series([0]*len(stock_ftw_plot))) == -1]

    fig, axs = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    sns.set_style("whitegrid")

    # Top subplot: Close price + buy/sell signals
    axs[0].plot(stock_ftw_plot.index, stock_ftw_plot['Close'], label='Close Price', color='blue', alpha=0.7)
    axs[0].scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy', s=80)
    axs[0].scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell', s=80)
    axs[0].set_ylabel("Price")
    axs[0].set_title(f"{ticker} Close Price & 52-Week High/Low Breakout Signals")
    axs[0].legend()
    axs[0].grid(True, linestyle='--', alpha=0.5)

    # Bottom subplot: 52-week high/low bands
    axs[1].plot(stock_ftw_plot.index, stock_ftw_plot['52w_high'], label='52-Week High', color='orange', linestyle='--')
    axs[1].plot(stock_ftw_plot.index, stock_ftw_plot['52w_low'], label='52-Week Low', color='green', linestyle='--')
    axs[1].set_ylabel("Band Value")
    axs[1].set_xlabel("Date")
    axs[1].set_title(f"{ticker} 52-Week High/Low Bands")
    axs[1].legend()
    axs[1].grid(True, linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.show()

In [ ]:
tickers = list(ftw_dict.keys())
n_tickers = len(tickers)

sns.set_style("whitegrid")
fig, axes = plt.subplots(n_tickers * 2, 1, figsize=(18, 5 * n_tickers), sharex=False)
axes = axes.flatten()

for i, ticker in enumerate(tickers):
    stock = ftw_dict[ticker].copy()
    buy_signals = stock[stock['Position'] == 1]
    sell_signals = stock[stock['Position'] == -1]

    ax_top = axes[i * 2]
    ax_bottom = axes[i * 2 + 1]

    # Top: Close price + buy/sell signals
    ax_top.plot(stock.index, stock['Close'], label='Close Price', color='blue', alpha=0.7)
    ax_top.scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy', s=80)
    ax_top.scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell', s=80)
    ax_top.set_ylabel("Price")
    ax_top.set_title(f"{ticker} 52-Week High/Low Breakout Trades")
    ax_top.legend()
    ax_top.grid(True, linestyle='--', alpha=0.5)

    # Bottom: 52-week high/low bands only
    ax_bottom.plot(stock.index, stock['52w_high'], label='52-Week High', color='orange', linestyle='--')
    ax_bottom.plot(stock.index, stock['52w_low'], label='52-Week Low', color='green', linestyle='--')
    ax_bottom.set_ylabel("Band Value")
    ax_bottom.set_xlabel("Date")
    ax_bottom.set_title(f"{ticker} 52-Week High/Low Bands")
    ax_bottom.legend()
    ax_bottom.grid(True, linestyle='--', alpha=0.5)

plt.suptitle('52-Week High/Low Breakout Strategy - All Tickers', fontsize=18, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# sorting values
ftw_metrics_by_year.sort_values(by=['Ticker', 'Year'], inplace=True)
ftw_metrics_by_year.head()

In [ ]:
#calculate overall metrics for each ticker for breakout strategy
metrics_rows_ftw = []
for ticker in holdings:
    ticker_trades_ftw = ftw_all_trades_df[ftw_all_trades_df['Ticker'] == ticker]
    metrics_row_ftw = calculate_metrics(ticker_trades_ftw)
    metrics_rows_ftw.append(metrics_row_ftw)
metrics_df_ftw = pd.concat(metrics_rows_ftw, ignore_index=True)

# 9. Display or plot results as needed
print(metrics_df_ftw.head())
print(metrics_df_ftw.head())

In [ ]:
# clean up and ensure correct datatypes for the metrics dataframe
metrics_df_ftw = metrics_df_ftw.dropna(subset=['Ticker'])
metrics_df_ftw['Ticker'] = metrics_df_ftw['Ticker'].astype(str)

In [ ]:
# overall metrics plot
fig, axs = plt.subplots(3, 3, figsize=(18, 14))
fig.suptitle('52-Week High/Low Breakout Strategy Metrics', fontsize=16)

# Cumulative Return by Ticker
axs[0, 0].bar(metrics_df_ftw['Ticker'], metrics_df_ftw['Cumulative Return'], color='mediumseagreen')
axs[0, 0].set_title('Cumulative Return')
axs[0, 0].set_xlabel('Ticker')
axs[0, 0].set_ylabel('Cumulative Return')
axs[0, 0].grid(axis='y', linestyle='--', alpha=0.7)

# CAGR by Ticker
axs[0, 1].bar(metrics_df_ftw['Ticker'], metrics_df_ftw['CAGR'], color='royalblue')
axs[0, 1].set_title('CAGR')
axs[0, 1].set_xlabel('Ticker')
axs[0, 1].set_ylabel('CAGR')
axs[0, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Total Return by Ticker
axs[0, 2].bar(metrics_df_ftw['Ticker'], metrics_df_ftw['Total Return'], color='seagreen')
axs[0, 2].set_title('Total Return')
axs[0, 2].set_xlabel('Ticker')
axs[0, 2].set_ylabel('Total Return')
axs[0, 2].grid(axis='y', linestyle='--', alpha=0.7)

# Standard Return by Ticker
axs[1, 0].bar(metrics_df_ftw['Ticker'], metrics_df_ftw['Standard Return'], color='slateblue')
axs[1, 0].set_title('Standard Return')
axs[1, 0].set_xlabel('Ticker')
axs[1, 0].set_ylabel('Standard Return')
axs[1, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Volatility by Ticker
axs[1, 1].bar(metrics_df_ftw['Ticker'], metrics_df_ftw['Volatility'], color='slateblue')
axs[1, 1].set_title('Volatility')       
axs[1, 1].set_xlabel('Ticker')
axs[1, 1].set_ylabel('Volatility')
axs[1, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Sharpe Ratio by Ticker
axs[1, 2].bar(metrics_df_ftw['Ticker'], metrics_df_ftw['Sharpe Ratio'], color='orange')
axs[1, 2].set_title('Sharpe Ratio')
axs[1, 2].set_xlabel('Ticker')
axs[1, 2].set_ylabel('Sharpe Ratio')
axs[1, 2].grid(axis='y', linestyle='--', alpha=0.7)

# Max Drawdown by Ticker
axs[2, 0].bar(metrics_df_ftw['Ticker'], metrics_df_ftw['Max Drawdown'], color='crimson')
axs[2, 0].set_title('Max Drawdown')
axs[2, 0].set_xlabel('Ticker')
axs[2, 0].set_ylabel('Max Drawdown')
axs[2, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Win Rate (%) by Ticker
axs[2, 1].bar(metrics_df_ftw['Ticker'], metrics_df_ftw['Win Rate (%)'], color='purple')
axs[2, 1].set_title('Win Rate (%)')
axs[2, 1].set_xlabel('Ticker')
axs[2, 1].set_ylabel('Win Rate (%)')
axs[2, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Profit Factor by Ticker
axs[2, 2].bar(metrics_df_ftw['Ticker'], metrics_df_ftw['Profit Factor'], color='teal')
axs[2, 2].set_title('Profit Factor')
axs[2, 2].set_xlabel('Ticker')
axs[2, 2].set_ylabel('Profit Factor')
axs[2, 2].grid(axis='y', linestyle='--', alpha=0.7)

In [ ]:

# 1. Calculate average annual return for your MA strategy
ftw_annual_return = ftw_metrics_by_year.groupby('Year')['Total Return'].mean()

In [ ]:

# 2. Calculate annual return for the mutual fund
fund_data['Fund Return'] = fund_data['Close'].pct_change()
fund_data = fund_data.dropna(subset=['Fund Return'])
fund_data['Year'] = fund_data.index.year
fund_annual_return = fund_data.groupby('Year')['Fund Return'].apply(lambda x: (1 + x).prod() - 1)

In [ ]:
# 3. Plot comparison
plt.figure(figsize=(12, 6))
plt.bar(ftw_annual_return.index - 0.15, ftw_annual_return.values, width=0.3, label='FTW Strategy Avg Return', color='skyblue')
plt.bar(fund_annual_return.index + 0.15, fund_annual_return.values, width=0.3, label='Mutual Fund Return', color='orange')
plt.xlabel('Year')
plt.ylabel('Annual Return')
plt.title('Annual Return: FTW Strategy (Avg) vs. Mutual Fund')
plt.legend()
plt.xticks(sorted(set(ftw_annual_return.index) | set(fund_annual_return.index)), rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# --- Summary Table: FTW Strategy (Avg) vs. Mutual Fund ---

# 1. FTW Strategy (average across tickers)
ftw_summary = pd.DataFrame({
    'Strategy': ['FTW Strategy (Avg)'],
    'CAGR': [ftw_metrics_df['CAGR'].mean()],
    'Total Return': [ftw_metrics_df['Total Return'].mean()],
    'Volatility': [ftw_metrics_df['Volatility'].mean()],
    'Max Drawdown': [ftw_metrics_df['Max Drawdown'].mean()],
    'Sharpe Ratio': [ftw_metrics_df['Sharpe Ratio'].mean()],
    'Win Rate (%)': [ftw_metrics_df['Win Rate (%)'].mean()],
    'Profit Factor': [ftw_metrics_df['Profit Factor'].mean()]
})

display(ftw_summary)

In [ ]:
# bollinger vs mutual fund summary table
ftw_summary_table = pd.concat([ftw_summary, fund_summary], ignore_index=True)
ftw_summary_table.set_index('Strategy', inplace=True)
display(ftw_summary_table)

**Relative Strength Momentum:**  
Computes RSI. Buys when RSI < 30, sells when RSI > 70. Trades and metrics are calculated.

In [ ]:
#calculate rate of change and trades for each stock
rel_all_trades = []
rel_dict = {}

for ticker in holdings:
    print(f"Processing {ticker} for RSI momentum strategy...")
    try:
        stock_rel = data[ticker][['Close']].dropna().copy()
        # Calculate 14-day RSI
        delta = stock_rel['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        stock_rel['RSI'] = 100 - (100 / (1 + rs))
        
        # Define RSI strategy: Buy when RSI < 30, Sell when RSI > 70
        stock_rel['Position'] = 0
        stock_rel.loc[stock_rel['RSI'] < 30, 'Position'] = 1
        stock_rel.loc[stock_rel['RSI'] > 70, 'Position'] = -1

        rel_dict[ticker] = stock_rel[['Close', 'RSI','Position']].copy()
        trades = calculate_trades(stock_rel)
        if not trades.empty:
            trades['Ticker'] = ticker
            rel_all_trades.append(trades)
    except KeyError as e:
        print(f"Error processing {ticker}: {e}")
        continue

if rel_all_trades:
    rel_all_trades_df = pd.concat(rel_all_trades, ignore_index=True)
    # Convert dates and add year column
    if 'Sell Date' in rel_all_trades_df.columns:
        rel_all_trades_df['Sell Date'] = pd.to_datetime(rel_all_trades_df['Sell Date'], format='%d-%m-%Y', errors='coerce')
        rel_all_trades_df['Buy Date'] = pd.to_datetime(rel_all_trades_df['Buy Date'], format='%d-%m-%Y', errors='coerce')
        rel_all_trades_df['Year'] = rel_all_trades_df['Sell Date'].dt.year
else:
    rel_all_trades_df = pd.DataFrame()  # Empty DataFrame if no trades

rel_all_trades_df.head()

In [ ]:
#group by ticker and year to calculate annual metrics for each group
rel_metrics_by_year = rel_all_trades_df.groupby(['Ticker', 'Year']).apply(calculate_metrics, risk_free_rate=0.02).reset_index(drop=True)

#Calculate overall metrics for each ticker
rel_metrics_rows = []
for ticker in holdings:
    ticker_trades = rel_all_trades_df[rel_all_trades_df['Ticker'] == ticker]
    rel_metrics_row = calculate_metrics(ticker_trades)
    rel_metrics_rows.append(rel_metrics_row)
rel_metrics_df = pd.concat(rel_metrics_rows, ignore_index=True)

# 9. Display or plot results as needed
print(rel_metrics_by_year.head())
print(rel_metrics_df.head())

In [ ]:
# sorting values
rel_metrics_by_year.sort_values(by=['Ticker', 'Year'], inplace=True)
rel_metrics_by_year.head()

In [ ]:
# Total returns pivot table for RSI strategy
total_return_rel = rel_metrics_by_year.pivot(index='Ticker', columns='Year', values='Total Return')
total_return_rel


In [ ]:
# Relative Strength strategy
rel_pivot = rel_metrics_by_year.groupby('Year')['Total Return'].mean().to_frame().T
rel_pivot.index = ['Relative Strength Strategy']
rel_pivot

In [ ]:
# Plot RSI momentum signals for each ticker
for ticker in rel_dict:
    stock_rel_plot = rel_dict[ticker].copy()
    buy_signals = stock_rel_plot[stock_rel_plot['Position'] == 1]
    sell_signals = stock_rel_plot[stock_rel_plot['Position'] == -1]

    fig, axs = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    sns.set_style("whitegrid")

    # Top subplot: Close price + buy/sell signals
    axs[0].plot(stock_rel_plot.index, stock_rel_plot['Close'], label='Close Price', color='blue', alpha=0.7)
    axs[0].scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy', s=80)
    axs[0].scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell', s=80)
    axs[0].set_ylabel("Price")
    axs[0].set_title(f"{ticker} Close Price & RSI Momentum Buy/Sell Signals")
    axs[0].legend()
    axs[0].grid(True, linestyle='--', alpha=0.5)

    # Bottom subplot: RSI values only
    axs[1].plot(stock_rel_plot.index, stock_rel_plot['RSI'], label='RSI', color='purple', alpha=0.7)
    axs[1].axhline(30, color='green', linestyle='--', label='Oversold (30)')
    axs[1].axhline(70, color='red', linestyle='--', label='Overbought (70)')
    axs[1].set_ylabel("RSI")
    axs[1].set_xlabel("Date")
    axs[1].set_title(f"{ticker} RSI Values")
    axs[1].legend()
    axs[1].grid(True, linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.show()

In [ ]:
tickers = list(rel_dict.keys())
n_tickers = len(tickers)

sns.set_style("whitegrid")
fig, axes = plt.subplots(n_tickers * 2, 1, figsize=(18, 5 * n_tickers), sharex=False)
axes = axes.flatten()

for i, ticker in enumerate(tickers):
    stock = rel_dict[ticker].copy()
    buy_signals = stock[stock['Position'] == 1]
    sell_signals = stock[stock['Position'] == -1]

    ax_top = axes[i * 2]
    ax_bottom = axes[i * 2 + 1]

    # Top: Close price + buy/sell signals
    ax_top.plot(stock.index, stock['Close'], label='Close Price', color='blue', alpha=0.7)
    ax_top.scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy', s=80)
    ax_top.scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell', s=80)
    ax_top.set_ylabel("Price")
    ax_top.set_title(f"{ticker} RSI Momentum Buy/Sell Signals")
    ax_top.legend()
    ax_top.grid(True, linestyle='--', alpha=0.5)

    # Bottom: RSI values only
    ax_bottom.plot(stock.index, stock['RSI'], label='RSI', color='purple', alpha=0.7)
    ax_bottom.axhline(30, color='green', linestyle='--', label='Oversold (30)')
    ax_bottom.axhline(70, color='red', linestyle='--', label='Overbought (70)')
    ax_bottom.set_ylabel("RSI")
    ax_bottom.set_xlabel("Date")
    ax_bottom.set_title(f"{ticker} RSI Values")
    ax_bottom.legend()
    ax_bottom.grid(True, linestyle='--', alpha=0.5)

plt.suptitle('RSI Momentum Strategy - All Tickers', fontsize=18, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# overall metrics plot
fig, axs = plt.subplots(3, 3, figsize=(18, 14))
fig.suptitle('RSI Momentum Strategy Metrics', fontsize=16)

# Cumulative Return by Ticker
axs[0, 0].bar(rel_metrics_df['Ticker'], rel_metrics_df['Cumulative Return'], color='mediumseagreen')
axs[0, 0].set_title('Cumulative Return')
axs[0, 0].set_xlabel('Ticker')
axs[0, 0].set_ylabel('Cumulative Return')
axs[0, 0].grid(axis='y', linestyle='--', alpha=0.7)

# CAGR by Ticker
axs[0, 1].bar(rel_metrics_df['Ticker'], rel_metrics_df['CAGR'], color='royalblue')
axs[0, 1].set_title('CAGR')
axs[0, 1].set_xlabel('Ticker')
axs[0, 1].set_ylabel('CAGR')
axs[0, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Total Return by Ticker
axs[0, 2].bar(rel_metrics_df['Ticker'], rel_metrics_df['Total Return'], color='seagreen')
axs[0, 2].set_title('Total Return')
axs[0, 2].set_xlabel('Ticker')
axs[0, 2].set_ylabel('Total Return')
axs[0, 2].grid(axis='y', linestyle='--', alpha=0.7)

# Standard Return by Ticker
axs[1, 0].bar(rel_metrics_df['Ticker'], rel_metrics_df['Standard Return'], color='slateblue')
axs[1, 0].set_title('Standard Return')
axs[1, 0].set_xlabel('Ticker')
axs[1, 0].set_ylabel('Standard Return')
axs[1, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Volatility by Ticker
axs[1, 1].bar(rel_metrics_df['Ticker'], rel_metrics_df['Volatility'], color='slateblue')
axs[1, 1].set_title('Volatility')
axs[1, 1].set_xlabel('Ticker')
axs[1, 1].set_ylabel('Volatility')
axs[1, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Sharpe Ratio by Ticker
axs[1, 2].bar(rel_metrics_df['Ticker'], rel_metrics_df['Sharpe Ratio'], color='orange')
axs[1, 2].set_title('Sharpe Ratio')
axs[1, 2].set_xlabel('Ticker')
axs[1, 2].set_ylabel('Sharpe Ratio')
axs[1, 2].grid(axis='y', linestyle='--', alpha=0.7)

# Max Drawdown by Ticker
axs[2, 0].bar(rel_metrics_df['Ticker'], rel_metrics_df['Max Drawdown'], color='crimson')
axs[2, 0].set_title('Max Drawdown')
axs[2, 0].set_xlabel('Ticker')
axs[2, 0].set_ylabel('Max Drawdown')
axs[2, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Win Rate (%) by Ticker
axs[2, 1].bar(rel_metrics_df['Ticker'], rel_metrics_df['Win Rate (%)'], color='purple')
axs[2, 1].set_title('Win Rate (%)')
axs[2, 1].set_xlabel('Ticker')
axs[2, 1].set_ylabel('Win Rate (%)')
axs[2, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Profit Factor by Ticker
axs[2, 2].bar(rel_metrics_df['Ticker'], rel_metrics_df['Profit Factor'], color='teal')
axs[2, 2].set_title('Profit Factor')
axs[2, 2].set_xlabel('Ticker')
axs[2, 2].set_ylabel('Profit Factor')
axs[2, 2].grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()



In [ ]:
# 1. Calculate average annual return for your MA strategy
rel_annual_return = rel_metrics_by_year.groupby('Year')['Total Return'].mean()

In [ ]:

# 2. Calculate annual return for the mutual fund
fund_data['Fund Return'] = fund_data['Close'].pct_change()
fund_data = fund_data.dropna(subset=['Fund Return'])
fund_data['Year'] = fund_data.index.year
fund_annual_return = fund_data.groupby('Year')['Fund Return'].apply(lambda x: (1 + x).prod() - 1)

In [ ]:
# 3. Plot comparison
plt.figure(figsize=(12, 6))
plt.bar(rel_annual_return.index - 0.15, rel_annual_return.values, width=0.3, label='RSI Strategy Avg Return', color='skyblue')
plt.bar(fund_annual_return.index + 0.15, fund_annual_return.values, width=0.3, label='Mutual Fund Return', color='orange')
plt.xlabel('Year')
plt.ylabel('Annual Return')
plt.title('Annual Return: RSI Strategy (Avg) vs. Mutual Fund')
plt.legend()
plt.xticks(sorted(set(rel_annual_return.index) | set(fund_annual_return.index)), rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
# Summary Table: RSI Momentum Strategy vs. Mutual Fund ---


# RSI Strategy (average across tickers)
rel_summary = pd.DataFrame({
    'Strategy': ['RSI Strategy (Avg)'],
    'CAGR': [rel_metrics_df['CAGR'].mean()],
    'Total Return': [rel_metrics_df['Total Return'].mean()],
    'Volatility': [rel_metrics_df['Volatility'].mean()],
    'Max Drawdown': [rel_metrics_df['Max Drawdown'].mean()],
    'Sharpe Ratio': [rel_metrics_df['Sharpe Ratio'].mean()],
    'Win Rate (%)': [rel_metrics_df['Win Rate (%)'].mean()],
    'Profit Factor': [rel_metrics_df['Profit Factor'].mean()]
})
display(rel_summary)


In [ ]:
# Combine and display
rel_summary_table = pd.concat([rel_summary, fund_summary], ignore_index=True)
rel_summary_table.set_index('Strategy', inplace=True)
display(rel_summary_table)

In [ ]:
#combine strategy metrics for comparison
momentum_startegies_summary = pd.concat([ftw_summary, roc_summary, rel_summary], ignore_index=True)
momentum_startegies_summary.set_index('Strategy', inplace=True)
display(momentum_startegies_summary)

In [ ]:
# Combine all strategy pivots into one DataFrame
combined_pivot_momentum = pd.concat([roc_pivot, ftw_pivot, rel_pivot], axis=0)
display(combined_pivot_momentum)

### Hydrid MultiIndicator

**Hybrid 1 (MA + RSI):**  
Combines MA and RSI. Buys when price is above MA50 and RSI < 30, sells when price is below MA200 and RSI > 70. Trades and metrics are calculated.


In [ ]:

hybrid1_trades = []
hybrid1_dict = {}

for ticker in holdings:
    print(f"Processing {ticker} for Hybrid Strategy 1...")
    try:
        stock_hybrid1 = data[ticker][['Close']].dropna().copy()
        # MA
        stock_hybrid1['MA50'] = stock_hybrid1['Close'].rolling(window=50).mean()
        stock_hybrid1['MA200'] = stock_hybrid1['Close'].rolling(window=200).mean()

        # RSI
        delta = stock_hybrid1['Close'].diff()
        gain  = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss  = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss.replace(0, np.nan)
        stock_hybrid1['RSI'] = 100 - (100 / (1 + rs))
        stock_hybrid1['RSI'] = stock_hybrid1['RSI'].fillna(0)

        # Hybrid signals
        stock_hybrid1['Position'] = 0
        stock_hybrid1.loc[(stock_hybrid1['Close'] > stock_hybrid1['MA50']) & (stock_hybrid1['RSI'] < 30), 'Position'] = 1
        stock_hybrid1.loc[(stock_hybrid1['Close'] < stock_hybrid1['MA200']) & (stock_hybrid1['RSI'] > 70), 'Position'] = -1

        #Hold logic
        stock_hybrid1['Hold Position'] = 0
        current_position = 0
        for idx,row in stock_hybrid1.iterrows():
            if row['Position'] == 1:
                current_position = 1
            elif row['Position'] == -1:
                current_position = 0
            stock_hybrid1.at[idx,'Hold Position'] = current_position
        
        stock_hybrid1['Prev_Hold'] = stock_hybrid1['Hold Position'].shift(1).fillna(0)
        stock_hybrid1['Pos_Marker'] = 0
        stock_hybrid1.loc[(stock_hybrid1['Prev_Hold'] != 1) & (stock_hybrid1['Hold Position'] == 1), 'Pos_Marker'] = 1
        stock_hybrid1.loc[(stock_hybrid1['Prev_Hold'] == 1) & (stock_hybrid1['Hold Position'] != 1), 'Pos_Marker'] = -1

        # Trades

        trades_hydrid1 = calculate_trades(pd.DataFrame({'Close': stock_hybrid1['Close'], 'Position': stock_hybrid1['Pos_Marker']}, index=stock_hybrid1.index))
        if not trades_hydrid1.empty:
            trades_hydrid1['Ticker'] = ticker
            hybrid1_trades.append(trades_hydrid1)
        hybrid1_dict[ticker] = stock_hybrid1[['Close','MA50','MA200','RSI','Position','Hold Position','Prev_Hold','Pos_Marker']].copy()
    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        continue

if hybrid1_trades:
    hybrid1_trades_df = pd.concat(hybrid1_trades, ignore_index=True)

    # convert dates and add year column
    hybrid1_trades_df['Sell Date'] = pd.to_datetime(hybrid1_trades_df['Sell Date'], format='%d-%m-%Y', errors='coerce')
    hybrid1_trades_df['Buy Date'] = pd.to_datetime(hybrid1_trades_df['Buy Date'], format='%d-%m-%Y', errors='coerce')
    hybrid1_trades_df['Year'] = hybrid1_trades_df['Sell Date'].dt.year

else:
    hybrid1_trades_df = pd.DataFrame()  # Empty DataFrame if no trades



In [ ]:
# Plot Hybrid Strategy 1 
for ticker in hybrid1_dict:
    stock_hybrid1_plot = hybrid1_dict[ticker].copy()
    buy_signals = stock_hybrid1_plot[stock_hybrid1_plot['Pos_Marker'] == 1]
    sell_signals = stock_hybrid1_plot[stock_hybrid1_plot['Pos_Marker'] == -1]

    fig, axs = plt.subplots(3, 1, figsize=(14, 12), sharex=True)
    sns.set_style("whitegrid")

    # Top subplot: Close price + buy/sell signals
    axs[0].plot(stock_hybrid1_plot.index, stock_hybrid1_plot['Close'], label='Close Price', color='blue', alpha=0.7)
    axs[0].scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy', s=80)
    axs[0].scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell', s=80)
    axs[0].set_ylabel("Price")
    axs[0].set_title(f"{ticker} Close Price & Hybrid Strategy Buy/Sell Signals")
    axs[0].legend()
    axs[0].grid(True, linestyle='--', alpha=0.5)

    # Middle subplot: MA20 and MA50
    axs[1].plot(stock_hybrid1_plot.index, stock_hybrid1_plot['MA50'], label='MA50', color='orange')
    axs[1].plot(stock_hybrid1_plot.index, stock_hybrid1_plot['MA200'], label='MA200', color='green')
    axs[1].set_ylabel("Moving Averages")
    axs[1].set_title(f"{ticker} MA50 and MA200")
    axs[1].legend()
    axs[1].grid(True, linestyle='--', alpha=0.5)

    # Bottom subplot: RSI values
    axs[2].plot(stock_hybrid1_plot.index, stock_hybrid1_plot['RSI'], label='RSI', color='purple')
    axs[2].axhline(30, color='green', linestyle='--', label='Oversold (30)')
    axs[2].axhline(70, color='red', linestyle='--', label='Overbought (70)')
    axs[2].set_ylabel("RSI")
    axs[2].set_title(f"{ticker} RSI")
    axs[2].legend()
    axs[2].grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()
    

In [ ]:
tickers = list(hybrid1_dict.keys())
n_tickers = len(tickers)

sns.set_style("whitegrid")
fig, axes = plt.subplots(n_tickers * 3, 1, figsize=(18, 7 * n_tickers), sharex=False)
axes = axes.flatten()

for i, ticker in enumerate(tickers):
    stock = hybrid1_dict[ticker].copy()
    buy_signals = stock[stock['Pos_Marker'] == 1]
    sell_signals = stock[stock['Pos_Marker'] == -1]

    ax_top = axes[i * 3]
    ax_mid = axes[i * 3 + 1]
    ax_bot = axes[i * 3 + 2]

    # Top: Close price + buy/sell signals
    ax_top.plot(stock.index, stock['Close'], label='Close Price', color='blue', alpha=0.7)
    ax_top.scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy', s=80)
    ax_top.scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell', s=80)
    ax_top.set_ylabel("Price")
    ax_top.set_title(f"{ticker} Hybrid1 (MA+RSI) Buy/Sell Signals")
    ax_top.legend()
    ax_top.grid(True, linestyle='--', alpha=0.5)

    # Middle: MA50 and MA200
    ax_mid.plot(stock.index, stock['MA50'], label='MA50', color='orange')
    ax_mid.plot(stock.index, stock['MA200'], label='MA200', color='green')
    ax_mid.set_ylabel("Moving Averages")
    ax_mid.set_title(f"{ticker} MA50 & MA200")
    ax_mid.legend()
    ax_mid.grid(True, linestyle='--', alpha=0.5)

    # Bottom: RSI values
    ax_bot.plot(stock.index, stock['RSI'], label='RSI', color='purple', alpha=0.7)
    ax_bot.axhline(30, color='green', linestyle='--', label='Oversold (30)')
    ax_bot.axhline(70, color='red', linestyle='--', label='Overbought (70)')
    ax_bot.set_ylabel("RSI")
    ax_bot.set_xlabel("Date")
    ax_bot.set_title(f"{ticker} RSI Values")
    ax_bot.legend()
    ax_bot.grid(True, linestyle='--', alpha=0.5)

plt.suptitle('Hybrid1 (MA + RSI) Strategy - All Tickers', fontsize=18, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# Metrics for Hybrid Strategy 1
metrics_hybrid1_by_year = hybrid1_trades_df.groupby(['Ticker', 'Year']).apply(calculate_metrics, risk_free_rate=0.02).reset_index(drop=True)

metrics_rows_hybrid1 = []

for ticker in holdings:
    ticker_trades_hybrid1 = hybrid1_trades_df[hybrid1_trades_df['Ticker'] == ticker]
    metrics_row_hybrid1 = calculate_metrics(ticker_trades_hybrid1)
    metrics_rows_hybrid1.append(metrics_row_hybrid1)
metrics_df_hybrid1 = pd.concat(metrics_rows_hybrid1, ignore_index=True)
print(metrics_df_hybrid1.head())

In [ ]:
#Calculate overall metrics for each ticker
hybrid1_metrics_rows = []
for ticker in holdings:
    hybrid1_ticker_trades = hybrid1_trades_df[hybrid1_trades_df['Ticker'] == ticker]
    hybrid1_metrics_row = calculate_metrics(hybrid1_ticker_trades)
    hybrid1_metrics_rows.append(hybrid1_metrics_row)
hybrid1_metrics_df = pd.concat(hybrid1_metrics_rows, ignore_index=True)

print(hybrid1_metrics_df.head())

In [ ]:
# sorting values
metrics_hybrid1_by_year.sort_values(by=['Ticker', 'Year'], inplace=True)
metrics_hybrid1_by_year.head()

In [ ]:
# Extract year from date for annual grouping
metrics_hybrid1_by_year['Year'] = metrics_hybrid1_by_year['Year']
annual_return = metrics_hybrid1_by_year.groupby(by='Year')['Standard Return'].prod()-1
# Plot annual returns as a bar char
annual_return.plot(kind='bar')
plt.show()

In [ ]:
# calculate pivot table for total returns by ticker and year
total_return_hybrid1 = metrics_hybrid1_by_year.pivot(index='Ticker', columns='Year', values='Total Return')
total_return_hybrid1

In [ ]:
# pivot table for Hybrid1 strategy
hybrid1_pivot = metrics_hybrid1_by_year.groupby('Year')['Total Return'].mean().to_frame().T
hybrid1_pivot.index = ['Hybrid1 Strategy']
hybrid1_pivot

In [ ]:
# Before plotting, cleaning the DataFrame:
hybrid1_metrics_df = hybrid1_metrics_df.dropna(subset=['Ticker'])
hybrid1_metrics_df['Ticker'] = hybrid1_metrics_df['Ticker'].astype(str)

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(18, 14))
fig.suptitle('Hybrid1 Strategy Metrics', fontsize=16)

# Cumulative Return by Ticker
axs[0, 0].bar(hybrid1_metrics_df['Ticker'], hybrid1_metrics_df['Cumulative Return'], color='mediumseagreen')
axs[0, 0].set_title('Cumulative Return')
axs[0, 0].set_xlabel('Ticker')
axs[0, 0].set_ylabel('Cumulative Return')
axs[0, 0].grid(axis='y', linestyle='--', alpha=0.7)

# CAGR by Ticker
axs[0, 1].bar(hybrid1_metrics_df['Ticker'], hybrid1_metrics_df['CAGR'], color='royalblue')
axs[0, 1].set_title('CAGR')
axs[0, 1].set_xlabel('Ticker')
axs[0, 1].set_ylabel('CAGR')
axs[0, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Total Return by Ticker
axs[0, 2].bar(hybrid1_metrics_df['Ticker'], hybrid1_metrics_df['Total Return'], color='seagreen')
axs[0, 2].set_title('Total Return')
axs[0, 2].set_xlabel('Ticker')
axs[0, 2].set_ylabel('Total Return')
axs[0, 2].grid(axis='y', linestyle='--', alpha=0.7)

# Standard Return by Ticker
axs[1, 0].bar(hybrid1_metrics_df['Ticker'], hybrid1_metrics_df['Standard Return'], color='slateblue')
axs[1, 0].set_title('Standard Return')
axs[1, 0].set_xlabel('Ticker')
axs[1, 0].set_ylabel('Standard Return')
axs[1, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Volatility by Ticker
axs[1, 1].bar(hybrid1_metrics_df['Ticker'], hybrid1_metrics_df['Volatility'], color='slateblue')
axs[1, 1].set_title('Volatility')
axs[1, 1].set_xlabel('Ticker')
axs[1, 1].set_ylabel('Volatility')
axs[1, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Sharpe Ratio by Ticker
axs[1, 2].bar(hybrid1_metrics_df['Ticker'], hybrid1_metrics_df['Sharpe Ratio'], color='orange')
axs[1, 2].set_title('Sharpe Ratio')
axs[1, 2].set_xlabel('Ticker')
axs[1, 2].set_ylabel('Sharpe Ratio')
axs[1, 2].grid(axis='y', linestyle='--', alpha=0.7)

# Max Drawdown by Ticker
axs[2, 0].bar(hybrid1_metrics_df['Ticker'], hybrid1_metrics_df['Max Drawdown'], color='crimson')
axs[2, 0].set_title('Max Drawdown')
axs[2, 0].set_xlabel('Ticker')
axs[2, 0].set_ylabel('Max Drawdown')
axs[2, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Win Rate (%) by Ticker
axs[2, 1].bar(hybrid1_metrics_df['Ticker'], hybrid1_metrics_df['Win Rate (%)'], color='purple')
axs[2, 1].set_title('Win Rate (%)')
axs[2, 1].set_xlabel('Ticker')
axs[2, 1].set_ylabel('Win Rate (%)')
axs[2, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Profit Factor by Ticker
axs[2, 2].bar(hybrid1_metrics_df['Ticker'], hybrid1_metrics_df['Profit Factor'], color='teal')
axs[2, 2].set_title('Profit Factor')
axs[2, 2].set_xlabel('Ticker')
axs[2, 2].set_ylabel('Profit Factor')
axs[2, 2].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
# Calculate average annual return for your Hybrid1MA strategy
hybrid1_annual_return = hybrid1_metrics_df.groupby('Year')['Total Return'].mean()

In [ ]:
# Calculate annual return for the mutual fund
fund_data['Fund Return'] = fund_data['Close'].pct_change()
fund_data = fund_data.dropna(subset=['Fund Return'])
fund_data['Year'] = fund_data.index.year
fund_annual_return = fund_data.groupby('Year')['Fund Return'].apply(lambda x: (1 + x).prod() - 1)

In [ ]:
#Plot comparison of annual returns

plt.figure(figsize=(12, 6))
plt.bar(hybrid1_annual_return.index - 0.15, hybrid1_annual_return.values, width=0.3, label='Hybrid1 Strategy Avg Return', color='skyblue')
plt.bar(fund_annual_return.index + 0.15, fund_annual_return.values, width=0.3, label='Mutual Fund Return', color='orange')
plt.xlabel('Year')
plt.ylabel('Annual Return')
plt.title('Annual Return: Hybrid1 Strategy (Avg) vs. Mutual Fund')
plt.legend()
plt.xticks(sorted(set(hybrid1_annual_return.index) | set(fund_annual_return.index)), rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# --- Summary Table: Hybrid1 Strategy (Avg) vs. Mutual Fund ---
# Hybrid1 Strategy (average across tickers)
hybrid1_summary = pd.DataFrame({
    'Strategy': ['Hybrid1 Strategy (Avg)'],
    'CAGR': [hybrid1_metrics_df['CAGR'].mean()],
    'Total Return': [hybrid1_metrics_df['Total Return'].mean()],
    'Volatility': [hybrid1_metrics_df['Volatility'].mean()],
    'Max Drawdown': [hybrid1_metrics_df['Max Drawdown'].mean()],
    'Sharpe Ratio': [hybrid1_metrics_df['Sharpe Ratio'].mean()],
    'Win Rate (%)': [hybrid1_metrics_df['Win Rate (%)'].mean()],
    'Profit Factor': [hybrid1_metrics_df['Profit Factor'].mean()]
})

In [ ]:
fund_summary = pd.DataFrame({
    'Strategy': ['Mutual Fund'],
    'CAGR': [fund_cagr],
    'Total Return': [fund_cum_return],
    'Volatility': [fund_volatility],
    'Max Drawdown': [fund_max_drawdown],
    'Sharpe Ratio': [fund_sharpe],
    'Win Rate (%)': [fund_win_rate],
    'Profit Factor': [fund_profit_factor]
})

In [ ]:
summary_table_hybrid1 = pd.concat([hybrid1_summary, fund_summary], ignore_index=True)
summary_table_hybrid1.set_index('Strategy', inplace=True)
display(summary_table_hybrid1)

**Hybrid 2 (MA + Bollinger Bands):**  
Combines MA and Bollinger Bands. Buys when price is above MA50 and below lower Bollinger band, sells when price is below MA200 and above upper Bollinger band. Trades and metrics are calculated.

In [ ]:
hybrid2_trades = []
hybrid2_dict = {}

for ticker in holdings:
    print(f"Processing {ticker} for Hybrid Strategy 2...")
    try:
        stock_hybrid2 = data[ticker][['Close']].dropna().copy()

        # MA
        stock_hybrid2['MA50'] = stock_hybrid2['Close'].rolling(window=50).mean()
        stock_hybrid2['MA200'] = stock_hybrid2['Close'].rolling(window=200).mean()

        # Bollinger Bands
        stock_hybrid2['BB_MA'] = stock_hybrid2['Close'].rolling(window=20).mean()
        stock_hybrid2['BB_STD'] = stock_hybrid2['Close'].rolling(window=20).std()
        stock_hybrid2['BB_Upper'] = stock_hybrid2['BB_MA'] + (2 * stock_hybrid2['BB_STD'])
        stock_hybrid2['BB_Lower'] = stock_hybrid2['BB_MA'] - (2 * stock_hybrid2['BB_STD'])

        # Hybrid signals
        stock_hybrid2['Position'] = 0
        stock_hybrid2.loc[(stock_hybrid2['Close'] > stock_hybrid2['MA50']) & (stock_hybrid2['Close'] < stock_hybrid2['BB_Lower']), 'Position'] = 1
        stock_hybrid2.loc[(stock_hybrid2['Close'] < stock_hybrid2['MA200']) & (stock_hybrid2['Close'] > stock_hybrid2['BB_Upper']), 'Position'] = -1

        # Hold logic
        stock_hybrid2['Hold Position'] = 0
        current_position = 0
        for idx,row in stock_hybrid2.iterrows():
            if row['Position'] == 1:
                current_position = 1
            elif row['Position'] == -1:
                current_position = 0
            stock_hybrid2.at[idx,'Hold Position'] = current_position
        stock_hybrid2['Prev_Hold'] = stock_hybrid2['Hold Position'].shift(1).fillna(0)
        stock_hybrid2['Pos_Marker'] = 0
        stock_hybrid2.loc[(stock_hybrid2['Prev_Hold'] != 1) & (stock_hybrid2['Hold Position'] == 1), 'Pos_Marker'] = 1
        stock_hybrid2.loc[(stock_hybrid2['Prev_Hold'] == 1) & (stock_hybrid2['Hold Position'] != 1), 'Pos_Marker'] = -1

        # Trades
        trades_hybrid2 = calculate_trades(pd.DataFrame({'Close': stock_hybrid2['Close'], 'Position': stock_hybrid2['Pos_Marker']}, index=stock_hybrid2.index))        
        if not trades_hybrid2.empty:
            trades_hybrid2['Ticker'] = ticker
            hybrid2_trades.append(trades_hybrid2)
        hybrid2_dict[ticker] = stock_hybrid2[['Close','MA50','MA200','BB_MA','BB_Upper','BB_Lower','Position','Hold Position','Prev_Hold','Pos_Marker']].copy()
    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        continue

if hybrid2_trades:
    hybrid2_trades_df = pd.concat(hybrid2_trades, ignore_index=True)
    columns = ['Ticker'] + [col for col in hybrid2_trades_df.columns if col != 'Ticker']
    hybrid2_trades_df = hybrid2_trades_df[columns]
    # convert dates and add year column
    hybrid2_trades_df['Sell Date'] = pd.to_datetime(hybrid2_trades_df['Sell Date'], format='%d-%m-%Y', errors='coerce')
    hybrid2_trades_df['Buy Date'] = pd.to_datetime(hybrid2_trades_df['Buy Date'], format='%d-%m-%Y', errors='coerce')
    hybrid2_trades_df['Year'] = hybrid2_trades_df['Sell Date'].dt.year
else:
    hybrid2_trades_df = pd.DataFrame()  # Empty DataFrame if no trades

In [ ]:
hybrid2_metrics_by_year = hybrid2_trades_df.groupby(['Ticker', 'Year']).apply(calculate_metrics, risk_free_rate=0.02).reset_index(drop=True)


In [ ]:
#Calculate overall metrics for each ticker
hybrid2_metrics_rows = []
for ticker in holdings:
    hybrid2_ticker_trades = hybrid2_trades_df[hybrid2_trades_df['Ticker'] == ticker]
    hybrid2_metrics_row = calculate_metrics(hybrid2_ticker_trades)
    hybrid2_metrics_rows.append(hybrid2_metrics_row)
hybrid2_metrics_df = pd.concat(hybrid2_metrics_rows, ignore_index=True)

# 9. Display or plot results as needed
print(hybrid2_metrics_by_year.head())
print(hybrid2_metrics_df.head())

In [ ]:
# sorting values
hybrid2_metrics_by_year.sort_values(by=['Ticker', 'Year'], inplace=True)
hybrid2_metrics_by_year.head()

In [ ]:
# Extract year from date for annual grouping
hybrid2_metrics_by_year['Year'] = hybrid2_metrics_by_year['Year']
annual_return = hybrid2_metrics_by_year.groupby(by='Year')['Standard Return'].prod()-1
# Plot annual returns as a bar char
annual_return.plot(kind='bar')
plt.show()

In [ ]:
# calculate pivot table for total returns by ticker and year
total_return_hybrid2 = hybrid2_metrics_by_year.pivot(index='Ticker', columns='Year', values='Total Return')
total_return_hybrid2

In [ ]:
hybrid2_pivot = hybrid2_metrics_by_year.groupby('Year')['Total Return'].mean().to_frame().T
hybrid2_pivot.index = ['Hybrid2 Strategy']
hybrid2_pivot

In [ ]:
# Bar plot: Total Return by Year (all tickers combined, mean)
total_return_by_year_hybrid2 = hybrid2_metrics_by_year.groupby('Year')['Total Return'].mean()
plt.figure(figsize=(10, 5))
plt.bar(total_return_by_year_hybrid2.index, total_return_by_year_hybrid2.values, color='skyblue')
plt.ylabel('Average Total Return')
plt.xlabel('Year')
plt.title('Average Total Return by Year (All Tickers)')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Before plotting, clean the DataFrame:
hybrid2_metrics_df = hybrid2_metrics_df.dropna(subset=['Ticker'])
hybrid2_metrics_df['Ticker'] = hybrid2_metrics_df['Ticker'].astype(str)

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(18, 14))
fig.suptitle('Hybrid2 Strategy Metrics', fontsize=16)

# Cumulative Return by Ticker
axs[0, 0].bar(hybrid2_metrics_df['Ticker'], hybrid2_metrics_df['Cumulative Return'], color='mediumseagreen')
axs[0, 0].set_title('Cumulative Return')
axs[0, 0].set_xlabel('Ticker')
axs[0, 0].set_ylabel('Cumulative Return')
axs[0, 0].grid(axis='y', linestyle='--', alpha=0.7)

# CAGR by Ticker
axs[0, 1].bar(hybrid2_metrics_df['Ticker'], hybrid2_metrics_df['CAGR'], color='royalblue')
axs[0, 1].set_title('CAGR')
axs[0, 1].set_xlabel('Ticker')
axs[0, 1].set_ylabel('CAGR')
axs[0, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Total Return by Ticker
axs[0, 2].bar(hybrid2_metrics_df['Ticker'], hybrid2_metrics_df['Total Return'], color='seagreen')
axs[0, 2].set_title('Total Return')
axs[0, 2].set_xlabel('Ticker')
axs[0, 2].set_ylabel('Total Return')
axs[0, 2].grid(axis='y', linestyle='--', alpha=0.7)

# Standard Return by Ticker
axs[1, 0].bar(hybrid2_metrics_df['Ticker'], hybrid2_metrics_df['Standard Return'], color='slateblue')
axs[1, 0].set_title('Standard Return')
axs[1, 0].set_xlabel('Ticker')
axs[1, 0].set_ylabel('Standard Return')
axs[1, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Volatility by Ticker
axs[1, 1].bar(hybrid2_metrics_df['Ticker'], hybrid2_metrics_df['Volatility'], color='slateblue')
axs[1, 1].set_title('Volatility')
axs[1, 1].set_xlabel('Ticker')
axs[1, 1].set_ylabel('Volatility')
axs[1, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Sharpe Ratio by Ticker
axs[1, 2].bar(hybrid2_metrics_df['Ticker'], hybrid2_metrics_df['Sharpe Ratio'], color='orange')
axs[1, 2].set_title('Sharpe Ratio')
axs[1, 2].set_xlabel('Ticker')
axs[1, 2].set_ylabel('Sharpe Ratio')
axs[1, 2].grid(axis='y', linestyle='--', alpha=0.7)

# Max Drawdown by Ticker
axs[2, 0].bar(hybrid2_metrics_df['Ticker'], hybrid2_metrics_df['Max Drawdown'], color='crimson')
axs[2, 0].set_title('Max Drawdown')
axs[2, 0].set_xlabel('Ticker')
axs[2, 0].set_ylabel('Max Drawdown')
axs[2, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Win Rate (%) by Ticker
axs[2, 1].bar(hybrid2_metrics_df['Ticker'], hybrid2_metrics_df['Win Rate (%)'], color='purple')
axs[2, 1].set_title('Win Rate (%)')
axs[2, 1].set_xlabel('Ticker')
axs[2, 1].set_ylabel('Win Rate (%)')
axs[2, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Profit Factor by Ticker
axs[2, 2].bar(hybrid2_metrics_df['Ticker'], hybrid2_metrics_df['Profit Factor'], color='teal')
axs[2, 2].set_title('Profit Factor')
axs[2, 2].set_xlabel('Ticker')
axs[2, 2].set_ylabel('Profit Factor')
axs[2, 2].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
# Plot Hybrid Strategy 2 Metrics
for ticker in hybrid2_dict:
    stock_hybrid2_plot = hybrid2_dict[ticker].copy()
    buy_signals = stock_hybrid2_plot[stock_hybrid2_plot['Pos_Marker'] == 1]
    sell_signals = stock_hybrid2_plot[stock_hybrid2_plot['Pos_Marker'] == -1]
    fig, axs = plt.subplots(2, 1, figsize=(14, 12), sharex=True)
    
    sns.set_style("whitegrid")
    axs[0].plot(stock_hybrid2_plot.index, stock_hybrid2_plot['Close'], label='Close Price', color='blue', alpha=0.7)
    axs[0].scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy', s=80)
    axs[0].scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell', s=80)
    axs[0].plot(stock_hybrid2_plot.index, stock_hybrid2_plot['MA50'], label='MA50', color='orange')
    axs[0].plot(stock_hybrid2_plot.index, stock_hybrid2_plot['MA200'], label='MA200', color='green')
    axs[0].set_ylabel("Price & MAs")
    axs[0].set_title(f"{ticker} Close Price, MAs & Hybrid Strategy Buy/Sell Signals")
    axs[0].legend()
    axs[0].grid(True, linestyle='--', alpha=0.5)
    
    axs[1].plot(stock_hybrid2_plot.index, stock_hybrid2_plot['BB_MA'], label='MA20 BB', color='purple')
    axs[1].plot(stock_hybrid2_plot.index, stock_hybrid2_plot['BB_Upper'], label='BB Upper', color='red', linestyle='--')
    axs[1].plot(stock_hybrid2_plot.index, stock_hybrid2_plot['BB_Lower'], label='BB Lower', color='green', linestyle='--')
    axs[1].set_ylabel("Bollinger Bands")
    axs[1].set_xlabel("Date")
    axs[1].legend()
    axs[1].grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()



In [ ]:

tickers = list(hybrid2_dict.keys())
n_tickers = len(tickers)

sns.set_style("whitegrid")
fig, axes = plt.subplots(n_tickers * 3, 1, figsize=(18, 7 * n_tickers), sharex=False)
axes = axes.flatten()

for i, ticker in enumerate(tickers):
    stock = hybrid2_dict[ticker].copy()
    buy_signals = stock[stock['Pos_Marker'] == 1]
    sell_signals = stock[stock['Pos_Marker'] == -1]

    ax_top = axes[i * 3]
    ax_mid = axes[i * 3 + 1]
    ax_bot = axes[i * 3 + 2]

    # Top: Close price + buy/sell signals
    ax_top.plot(stock.index, stock['Close'], label='Close Price', color='blue', alpha=0.7)
    ax_top.scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy', s=80)
    ax_top.scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell', s=80)
    ax_top.set_ylabel("Price")
    ax_top.set_title(f"{ticker} Hybrid2 (MA+Bollinger) Buy/Sell Signals")
    ax_top.legend()
    ax_top.grid(True, linestyle='--', alpha=0.5)

    # Middle: MA50 and MA200
    ax_mid.plot(stock.index, stock['MA50'], label='MA50', color='orange')
    ax_mid.plot(stock.index, stock['MA200'], label='MA200', color='green')
    ax_mid.set_ylabel("Moving Averages")
    ax_mid.set_title(f"{ticker} MA50 & MA200")
    ax_mid.legend()
    ax_mid.grid(True, linestyle='--', alpha=0.5)

    # Bottom: Bollinger Bands
    ax_bot.plot(stock.index, stock['BB_MA'], label='BB MA', color='purple')
    ax_bot.plot(stock.index, stock['BB_Upper'], label='BB Upper', color='red', linestyle='--')
    ax_bot.plot(stock.index, stock['BB_Lower'], label='BB Lower', color='green', linestyle='--')
    ax_bot.set_ylabel("Bollinger Bands")
    ax_bot.set_xlabel("Date")
    ax_bot.set_title(f"{ticker} Bollinger Bands")
    ax_bot.legend()
    ax_bot.grid(True, linestyle='--', alpha=0.5)

plt.suptitle('Hybrid2 (MA + Bollinger Band) Strategy - All Tickers', fontsize=18, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate average annual return for your MA strategy
hybrid2_annual_return = hybrid2_metrics_by_year.groupby('Year')['Total Return'].mean()

In [ ]:
# Calculate annual return for the mutual fund
fund_data['Fund Return'] = fund_data['Close'].pct_change()
fund_data = fund_data.dropna(subset=['Fund Return'])
fund_data['Year'] = fund_data.index.year
fund_annual_return = fund_data.groupby('Year')['Fund Return'].apply(lambda x: (1 + x).prod() - 1)

In [ ]:
# Plot comparison
plt.figure(figsize=(12, 6))
plt.bar(hybrid2_annual_return.index - 0.15, hybrid2_annual_return.values, width=0.3, label='Hybrid2 Strategy Avg Return', color='skyblue')
plt.bar(fund_annual_return.index + 0.15, fund_annual_return.values, width=0.3, label='Mutual Fund Return', color='orange')
plt.xlabel('Year')
plt.ylabel('Annual Return')
plt.title('Annual Return: Hybrid2 Strategy (Avg) vs. Mutual Fund')
plt.legend()
plt.xticks(sorted(set(hybrid2_annual_return.index) | set(fund_annual_return.index)), rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# --- Summary Table: Hybrid2 Strategy (Avg) vs. Mutual Fund ---

# Hybrid2 Strategy (average across tickers)
hybrid2_summary = pd.DataFrame({
    'Strategy': ['Hybrid2 Strategy (Avg)'],
    'CAGR': [hybrid2_metrics_df['CAGR'].mean()],
    'Total Return': [hybrid2_metrics_df['Total Return'].mean()],
    'Volatility': [hybrid2_metrics_df['Volatility'].mean()],
    'Max Drawdown': [hybrid2_metrics_df['Max Drawdown'].mean()],
    'Sharpe Ratio': [hybrid2_metrics_df['Sharpe Ratio'].mean()],
    'Win Rate (%)': [hybrid2_metrics_df['Win Rate (%)'].mean()],
    'Profit Factor': [hybrid2_metrics_df['Profit Factor'].mean()]
})

fund_summary = pd.DataFrame({
    'Strategy': ['Mutual Fund'],
    'CAGR': [fund_cagr],
    'Total Return': [fund_cum_return],
    'Volatility': [fund_volatility],
    'Max Drawdown': [fund_max_drawdown],
    'Sharpe Ratio': [fund_sharpe],
    'Win Rate (%)': [fund_win_rate],
    'Profit Factor': [fund_profit_factor]
})

In [ ]:

# Combine and display
summary_table_hybrid2 = pd.concat([hybrid2_summary, fund_summary], ignore_index=True)
summary_table_hybrid2.set_index('Strategy', inplace=True)
display(summary_table_hybrid2)

**Hybrid 3 (MA + RSI + Bollinger Bands):**  
Combines MA, RSI, and Bollinger Bands. Buys when price is above short MA, RSI < 30, and below lower Bollinger band; sells when price is below long MA, RSI > 70, and above upper Bollinger band. Trades and metrics are calculated.

In [ ]:
hybrid3_trades = []
hybrid3_dict = {}

for ticker in holdings:
    print(f"Processing {ticker} for Hybrid Strategy 3...")
    try:
        stock_hybrid3 = data[ticker][['Close']].dropna().copy()
        # MA
        stock_hybrid3['Short MA'] = stock_hybrid3['Close'].rolling(window=50).mean()
        stock_hybrid3['Long MA'] = stock_hybrid3['Close'].rolling(window=200).mean()

        # RSI
        delta = stock_hybrid3['Close'].diff()
        gain  = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss  = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss.replace(0, np.nan)
        stock_hybrid3['RSI'] = 100 - (100 / (1 + rs))
        stock_hybrid3['RSI'] = stock_hybrid3['RSI'].fillna(0)

        # Bollinger Bands
        stock_hybrid3['BB_MA'] = stock_hybrid3['Close'].rolling(window=20).mean()
        stock_hybrid3['BB_STD'] = stock_hybrid3['Close'].rolling(window=20).std()
        stock_hybrid3['BB_Upper'] = stock_hybrid3['BB_MA'] + (2 * stock_hybrid3['BB_STD'])
        stock_hybrid3['BB_Lower'] = stock_hybrid3['BB_MA'] - (2 * stock_hybrid3['BB_STD'])

        # Hybrid signals
        stock_hybrid3['Position'] = 0
        stock_hybrid3.loc[
            (stock_hybrid3['Close'] > stock_hybrid3['Short MA']) &
            (stock_hybrid3['RSI'] < 30) &
            (stock_hybrid3['Close'] < stock_hybrid3['BB_Lower']),
            'Position'] = 1
        stock_hybrid3.loc[
            (stock_hybrid3['Close'] < stock_hybrid3['Long MA']) &
            (stock_hybrid3['RSI'] > 70) &
            (stock_hybrid3['Close'] > stock_hybrid3['BB_Upper']),
            'Position'] = -1
        
        # Hold logic
        stock_hybrid3['Hold Position'] = 0
        current_position = 0
        for idx,row in stock_hybrid3.iterrows():
            if row['Position'] == 1:
                current_position = 1
            elif row['Position'] == -1:
                current_position = 0
            stock_hybrid3.at[idx,'Hold Position'] = current_position
        stock_hybrid3['Prev_Hold'] = stock_hybrid3['Hold Position'].shift(1).fillna(0)
        stock_hybrid3['Pos_Marker'] = 0
        stock_hybrid3.loc[(stock_hybrid3['Prev_Hold'] != 1) & (stock_hybrid3['Hold Position'] == 1), 'Pos_Marker'] = 1
        stock_hybrid3.loc[(stock_hybrid3['Prev_Hold'] == 1) & (stock_hybrid3['Hold Position'] != 1), 'Pos_Marker'] = -1

        # Trades
        trades_hybrid3 = calculate_trades(pd.DataFrame({'Close': stock_hybrid3['Close'], 'Position': stock_hybrid3['Pos_Marker']}, index=stock_hybrid3.index))
        if not trades_hybrid3.empty:
            trades_hybrid3['Ticker'] = ticker
            hybrid3_trades.append(trades_hybrid3)
        hybrid3_dict[ticker] = stock_hybrid3[['Close','Short MA','Long MA','RSI','BB_MA','BB_Upper','BB_Lower','Position','Hold Position','Prev_Hold','Pos_Marker']].copy()
    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        continue

if hybrid3_trades:
    hybrid3_trades_df = pd.concat(hybrid3_trades, ignore_index=True)

    # convert dates and add year column
    hybrid3_trades_df['Sell Date'] = pd.to_datetime(hybrid3_trades_df['Sell Date'], format='%d-%m-%Y', errors='coerce')
    hybrid3_trades_df['Buy Date'] = pd.to_datetime(hybrid3_trades_df['Buy Date'], format='%d-%m-%Y', errors='coerce')
    hybrid3_trades_df['Year'] = hybrid3_trades_df['Sell Date'].dt.year

else:
    hybrid3_trades_df = pd.DataFrame()  # Empty DataFrame if no trades

In [ ]:
# Plot Hybrid Strategy 3 Metrics
for ticker in hybrid3_dict:
    stock_hybrid3_plot = hybrid3_dict[ticker].copy()
    buy_signals = stock_hybrid3_plot[stock_hybrid3_plot['Pos_Marker'] == 1]
    sell_signals = stock_hybrid3_plot[stock_hybrid3_plot['Pos_Marker'] == -1]
    fig, axs = plt.subplots(3, 1, figsize=(14, 16), sharex=True)
    sns.set_style("whitegrid")
    axs[0].plot(stock_hybrid3_plot.index, stock_hybrid3_plot['Close'], label='Close Price', color='blue', alpha=0.7)
    axs[0].scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy', s=80)
    axs[0].scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell', s=80)
    axs[0].plot(stock_hybrid3_plot.index, stock_hybrid3_plot['Short MA'], label='Short MA', color='orange')
    axs[0].plot(stock_hybrid3_plot.index, stock_hybrid3_plot['Long MA'], label='Long MA', color='green')
    axs[0].set_ylabel("Price & MAs")
    axs[0].set_title(f"{ticker} Close Price, MAs & Hybrid Strategy Buy/Sell Signals")
    axs[0].legend()
    axs[0].grid(True, linestyle='--', alpha=0.5)

    axs[1].plot(stock_hybrid3_plot.index, stock_hybrid3_plot['RSI'], label='RSI', color='purple')
    axs[1].axhline(30, color='green', linestyle='--', label='Oversold (30)')
    axs[1].axhline(70, color='red', linestyle='--', label='Overbought (70)')
    axs[1].set_ylabel("RSI")
    axs[1].set_title(f"{ticker} RSI")
    axs[1].legend()
    axs[1].grid(True, linestyle='--', alpha=0.5)

    axs[2].plot(stock_hybrid3_plot.index, stock_hybrid3_plot['BB_MA'], label='BB MA', color='purple')
    axs[2].plot(stock_hybrid3_plot.index, stock_hybrid3_plot['BB_Upper'], label='BB Upper', color='red', linestyle='--')
    axs[2].plot(stock_hybrid3_plot.index, stock_hybrid3_plot['BB_Lower'], label='BB Lower', color='green', linestyle='--')
    axs[2].set_ylabel("Bollinger Bands")
    axs[2].set_xlabel("Date")
    axs[2].legend()
    axs[2].grid(True, linestyle='--', alpha=0.5)    
    plt.tight_layout()
    plt.show()

In [ ]:

tickers = list(hybrid2_dict.keys())
n_tickers = len(tickers)

sns.set_style("whitegrid")
fig, axes = plt.subplots(n_tickers * 3, 1, figsize=(18, 7 * n_tickers), sharex=False)
axes = axes.flatten()

for i, ticker in enumerate(tickers):
    stock = hybrid2_dict[ticker].copy()
    buy_signals = stock[stock['Pos_Marker'] == 1]
    sell_signals = stock[stock['Pos_Marker'] == -1]

    ax_top = axes[i * 3]
    ax_mid = axes[i * 3 + 1]
    ax_bot = axes[i * 3 + 2]

    # Top: Close price + buy/sell signals
    ax_top.plot(stock.index, stock['Close'], label='Close Price', color='blue', alpha=0.7)
    ax_top.scatter(buy_signals.index, buy_signals['Close'], marker='^', color='green', label='Buy', s=80)
    ax_top.scatter(sell_signals.index, sell_signals['Close'], marker='v', color='red', label='Sell', s=80)
    ax_top.set_ylabel("Price")
    ax_top.set_title(f"{ticker} Hybrid2 (MA+Bollinger) Buy/Sell Signals")
    ax_top.legend()
    ax_top.grid(True, linestyle='--', alpha=0.5)

    # Middle: MA50 and MA200
    ax_mid.plot(stock.index, stock['MA50'], label='MA50', color='orange')
    ax_mid.plot(stock.index, stock['MA200'], label='MA200', color='green')
    ax_mid.set_ylabel("Moving Averages")
    ax_mid.set_title(f"{ticker} MA20 & MA50")
    ax_mid.legend()
    ax_mid.grid(True, linestyle='--', alpha=0.5)

    # Bottom: Bollinger Bands
    ax_bot.plot(stock.index, stock['BB_MA'], label='BB MA', color='purple')
    ax_bot.plot(stock.index, stock['BB_Upper'], label='BB Upper', color='red', linestyle='--')
    ax_bot.plot(stock.index, stock['BB_Lower'], label='BB Lower', color='green', linestyle='--')
    ax_bot.set_ylabel("Bollinger Bands")
    ax_bot.set_xlabel("Date")
    ax_bot.set_title(f"{ticker} Bollinger Bands")
    ax_bot.legend()
    ax_bot.grid(True, linestyle='--', alpha=0.5)

plt.suptitle('Hybrid2 (MA + Bollinger Band) Strategy - All Tickers', fontsize=18, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
#group by ticker and year to calculate annual metrics for each group
hybrid3_metrics_by_year = hybrid3_trades_df.groupby(['Ticker', 'Year']).apply(calculate_metrics, risk_free_rate=0.02).reset_index(drop=True)


In [ ]:
#Calculate overall metrics for each ticker
hybrid3_metrics_rows = []
for ticker in holdings:
    hybrid3_ticker_trades = hybrid3_trades_df[hybrid3_trades_df['Ticker'] == ticker]
    hybrid3_metrics_row = calculate_metrics(hybrid3_ticker_trades)
    hybrid3_metrics_rows.append(hybrid3_metrics_row)
hybrid3_metrics_df = pd.concat(hybrid3_metrics_rows, ignore_index=True)

# Display or plot results as needed
print(hybrid3_metrics_by_year.head())
print(hybrid3_metrics_df.head())

In [ ]:
# sorting values
hybrid3_metrics_by_year.sort_values(by=['Ticker', 'Year'], inplace=True)
hybrid3_metrics_by_year.head()

In [ ]:
# Extract year from date for annual grouping
hybrid3_metrics_by_year['Year'] = hybrid3_metrics_by_year['Year']
annual_return = hybrid3_metrics_by_year.groupby(by='Year')['Standard Return'].prod()-1
# Plot annual returns as a bar char
annual_return.plot(kind='bar')
plt.show()

In [ ]:
# calculate pivot table for total returns by ticker and year
total_return_hybrid3 = hybrid3_metrics_by_year.pivot(index='Ticker', columns='Year', values='Total Return')
total_return_hybrid3

In [ ]:
# pivot table for Hybrid3 strategy
hybrid3_pivot = hybrid3_metrics_by_year.groupby('Year')['Total Return'].mean().to_frame().T
hybrid3_pivot.index = ['Hybrid3 Strategy']
hybrid3_pivot

In [ ]:
# Bar plot: Total Return by Year (all tickers combined, mean)
total_return_by_year_hybrid3 = hybrid3_metrics_by_year.groupby('Year')['Total Return'].mean()
plt.figure(figsize=(10, 5))
plt.bar(total_return_by_year_hybrid3.index, total_return_by_year_hybrid3.values, color='skyblue')
plt.ylabel('Average Total Return')
plt.xlabel('Year')
plt.title('Average Total Return by Year (All Tickers)')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Before plotting, clean the DataFrame:
hybrid3_metrics_df = hybrid3_metrics_df.dropna(subset=['Ticker'])
hybrid3_metrics_df['Ticker'] = hybrid3_metrics_df['Ticker'].astype(str)

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(18, 14))
fig.suptitle('Hybrid3 Strategy Metrics', fontsize=16)

# Cumulative Return by Ticker
axs[0, 0].bar(hybrid3_metrics_df['Ticker'], hybrid3_metrics_df['Cumulative Return'], color='mediumseagreen')
axs[0, 0].set_title('Cumulative Return')
axs[0, 0].set_xlabel('Ticker')
axs[0, 0].set_ylabel('Cumulative Return')
axs[0, 0].grid(axis='y', linestyle='--', alpha=0.7)

# CAGR by Ticker
axs[0, 1].bar(hybrid3_metrics_df['Ticker'], hybrid3_metrics_df['CAGR'], color='royalblue')
axs[0, 1].set_title('CAGR')
axs[0, 1].set_xlabel('Ticker')
axs[0, 1].set_ylabel('CAGR')
axs[0, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Total Return by Ticker
axs[0, 2].bar(hybrid3_metrics_df['Ticker'], hybrid3_metrics_df['Total Return'], color='seagreen')
axs[0, 2].set_title('Total Return')
axs[0, 2].set_xlabel('Ticker')
axs[0, 2].set_ylabel('Total Return')
axs[0, 2].grid(axis='y', linestyle='--', alpha=0.7)

# Standard Return by Ticker
axs[1, 0].bar(hybrid3_metrics_df['Ticker'], hybrid3_metrics_df['Standard Return'], color='slateblue')
axs[1, 0].set_title('Standard Return')
axs[1, 0].set_xlabel('Ticker')
axs[1, 0].set_ylabel('Standard Return')
axs[1, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Volatility by Ticker
axs[1, 1].bar(hybrid3_metrics_df['Ticker'], hybrid3_metrics_df['Volatility'], color='slateblue')
axs[1, 1].set_title('Volatility')
axs[1, 1].set_xlabel('Ticker')
axs[1, 1].set_ylabel('Volatility')
axs[1, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Sharpe Ratio by Ticker
axs[1, 2].bar(hybrid3_metrics_df['Ticker'], hybrid3_metrics_df['Sharpe Ratio'], color='orange')
axs[1, 2].set_title('Sharpe Ratio')
axs[1, 2].set_xlabel('Ticker')
axs[1, 2].set_ylabel('Sharpe Ratio')
axs[1, 2].grid(axis='y', linestyle='--', alpha=0.7)

# Max Drawdown by Ticker
axs[2, 0].bar(hybrid3_metrics_df['Ticker'], hybrid3_metrics_df['Max Drawdown'], color='crimson')
axs[2, 0].set_title('Max Drawdown')
axs[2, 0].set_xlabel('Ticker')
axs[2, 0].set_ylabel('Max Drawdown')
axs[2, 0].grid(axis='y', linestyle='--', alpha=0.7)

# Win Rate (%) by Ticker
axs[2, 1].bar(hybrid3_metrics_df['Ticker'], hybrid3_metrics_df['Win Rate (%)'], color='purple')
axs[2, 1].set_title('Win Rate (%)')
axs[2, 1].set_xlabel('Ticker')
axs[2, 1].set_ylabel('Win Rate (%)')
axs[2, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Profit Factor by Ticker
axs[2, 2].bar(hybrid3_metrics_df['Ticker'], hybrid3_metrics_df['Profit Factor'], color='teal')
axs[2, 2].set_title('Profit Factor')
axs[2, 2].set_xlabel('Ticker')
axs[2, 2].set_ylabel('Profit Factor')
axs[2, 2].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
# Calculate average annual return for your MA strategy
hybrid3_annual_return = hybrid3_metrics_by_year.groupby('Year')['Total Return'].mean()

In [ ]:
# 2. Calculate annual return for the mutual fund
fund_data['Fund Return'] = fund_data['Close'].pct_change()
fund_data = fund_data.dropna(subset=['Fund Return'])
fund_data['Year'] = fund_data.index.year
fund_annual_return = fund_data.groupby('Year')['Fund Return'].apply(lambda x: (1 + x).prod() - 1)

In [ ]:

# 3. Plot comparison
plt.figure(figsize=(12, 6))
plt.bar(hybrid3_annual_return.index - 0.15, hybrid3_annual_return.values, width=0.3, label='Hybrid3 Strategy Avg Return', color='skyblue')
plt.bar(fund_annual_return.index + 0.15, fund_annual_return.values, width=0.3, label='Mutual Fund Return', color='orange')
plt.xlabel('Year')
plt.ylabel('Annual Return')
plt.title('Annual Return: Hybrid3 Strategy (Avg) vs. Mutual Fund')
plt.legend()
plt.xticks(sorted(set(hybrid3_annual_return.index) | set(fund_annual_return.index)), rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
# Summary Table for Hybrid Strategies

hybrid3_summary = pd.DataFrame({
    'Strategy' : ['Hybrid MA+RSI'],
    'CAGR' : [hybrid3_metrics_df['CAGR'].mean()],
    'Total Return' : [hybrid3_metrics_df['Total Return'].mean()],
    'Volatility' : [hybrid3_metrics_df['Volatility'].mean()],
    'Max Drawdown' : [hybrid3_metrics_df['Max Drawdown'].mean()],
    'Sharpe Ratio' : [hybrid3_metrics_df['Sharpe Ratio'].mean()],
    'Win Rate (%)' : [hybrid3_metrics_df['Win Rate (%)'].mean()],
    'Profit Factor' : [hybrid3_metrics_df['Profit Factor'].mean()]
})

fund_summary = pd.DataFrame({
    'Strategy': ['Mutual Fund'],
    'CAGR': [fund_cagr],
    'Total Return': [fund_cum_return],
    'Volatility': [fund_volatility],
    'Max Drawdown': [fund_max_drawdown],
    'Sharpe Ratio': [fund_sharpe],
    'Win Rate (%)': [fund_win_rate],
    'Profit Factor': [fund_profit_factor]
})



In [ ]:
summary_table_hybrid3 = pd.concat([hybrid3_summary, fund_summary], ignore_index=True)
summary_table_hybrid3.set_index('Strategy', inplace=True)
display(summary_table_hybrid3)

In [ ]:
hybrid_strategies_summary = pd.concat([hybrid1_summary, hybrid2_summary, hybrid3_summary], ignore_index=True)
hybrid_strategies_summary.set_index('Strategy', inplace=True)
display(hybrid_strategies_summary)

In [ ]:
# combining all pivot tables
combined_pivot_hybrid = pd.concat([hybrid1_pivot, hybrid2_pivot, hybrid3_pivot])
combined_pivot_hybrid

In [ ]:
trend_startegies_summary.reset_index(inplace=True)
meanrev_startegies_summary.reset_index(inplace=True)
momentum_startegies_summary.reset_index(inplace=True)
hybrid_strategies_summary.reset_index(inplace=True)

In [ ]:
# combining all strategies summary
strategies_summary = pd.concat([trend_startegies_summary, meanrev_startegies_summary,momentum_startegies_summary ,hybrid_strategies_summary], ignore_index=True)
strategies_summary.set_index('Strategy', inplace=True)
display(strategies_summary)

In [ ]:
# combine all pivot tables for all strategies
combined_pivot_all = pd.concat([combined_pivot_ma, combined_pivot_meanrev, combined_pivot_momentum, combined_pivot_hybrid])

In [ ]:
# Add mutual fund annual returns as the last row
fund_data['Fund Return'] = fund_data['Close'].pct_change()
fund_data = fund_data.dropna(subset=['Fund Return'])
fund_data['Year'] = fund_data.index.year
fund_annual_return = fund_data.groupby('Year')['Fund Return'].apply(lambda x: (1 + x).prod() - 1)
mutual_fund_row = pd.DataFrame([fund_annual_return.values], columns=fund_annual_return.index, index=['Mutual Fund'])

combined_pivot_all_with_fund = pd.concat([combined_pivot_all, mutual_fund_row], sort=True)
display(combined_pivot_all_with_fund)


1. Most technical-based strategies delivered higher average return than the mutual funds in several years.
2. The mutual fund had steadier returns, but technical strategies often outperformed in bullish years and underperformed in bearish years.
3. Strategies such as RSI and Hybrid approaches generally had higher Sharpe ratios and win rates, suggesting better risk-adjusted performance.

Technical indicator-based strategies can outperform top-performing mutual funds in certain market conditions, but may be more volatile and require active management. For investors willing to accept higher risk and effort, these strategies offer the potential for superior returns.